In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Data Preparation

In [2]:
# import the datasets 

elections_1949_df = pd.read_csv('Bundestagswahlen/btw1949_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1953_df = pd.read_csv('Bundestagswahlen/btw1953_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1957_df = pd.read_csv('Bundestagswahlen/btw1957_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1961_df = pd.read_csv('Bundestagswahlen/btw1961_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1965_df = pd.read_csv('Bundestagswahlen/btw1965_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1969_df = pd.read_csv('Bundestagswahlen/btw1969_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1972_df = pd.read_csv('Bundestagswahlen/btw1972_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1976_df = pd.read_csv('Bundestagswahlen/btw1976_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1980_df = pd.read_csv('Bundestagswahlen/btw1980_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1983_df = pd.read_csv('Bundestagswahlen/btw1983_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1987_df = pd.read_csv('Bundestagswahlen/btw1987_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1990_df = pd.read_csv('Bundestagswahlen/btw1990_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1994_df = pd.read_csv('Bundestagswahlen/btw1994_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_1998_df = pd.read_csv('Bundestagswahlen/btw1998_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_2002_df = pd.read_csv('Bundestagswahlen/btw2002_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_2005_df = pd.read_csv('Bundestagswahlen/btw2005_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_2009_df = pd.read_csv('Bundestagswahlen/btw2009_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_2013_df = pd.read_csv('Bundestagswahlen/btw2013_kerg.csv', header=5, encoding='latin-1', sep=';')
elections_2017_df = pd.read_csv('Bundestagswahlen/btw2017_kerg.csv', header=5, sep=';')

Every year a different person makes these datasets and there is not clear format. Additionally, this person is no computer scientist. Therefore we will clean up the datasets according to the following rules:

1. The different header lines will be merged to become one
2. Empty rows that have been created for visibility will be cut
3. All parties that have in the span from 1949 to 2017 reached 5% or more in a federal election once get their own column, others will be grouped as "rest"
4. CSU are merged with CDU in all elections
5. The PDS is counted as the LINKE before they were founded
6. The KPD and DKP are merged as one party. 
7. Colums called "Vorperiode" (previous period) will be kept when possible since we could use them as a prior

In [3]:
print(elections_1949_df.columns)
col = elections_1949_df[elections_1949_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
print(col1.values)
print(valid_votes)
perc = col1.values/valid_votes
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Gültige', 'SPD', 'CDU', 'FDP', 'CSU', 'KPD', 'Parteilose',
       'BP', 'DP', 'Zentrum', 'WAV', 'DKP/DRP', 'RSF', 'SSW', 'EVD', 'RWVP',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38'],
      dtype='object')
[[3.1207620e+07 2.4495614e+07 7.6321600e+05 2.3732398e+07 6.9349750e+06
  5.9786360e+06 2.8299200e+06 1.3804480e+06 1.3617060e+06 1.1416470e+06
  9.8647800e+05 9.3993400e+05 7.2750500e+05 6.8188800e+05 4.2903100e+05
  2.1674900e+05 7.5388000e+04 2.6162000e+04 2.1931000e+04]]
[23732398.]


,Wahlberechtigte,Wähler,Ungültige,Gültige,SPD,CDU,FDP,CSU,KPD,Parteilose,BP,DP,Zentrum,WAV,DKP/DRP,RSF,SSW,EVD,RWVP
0,3.120762e+07,2.449561e+07,763216.000000,23732398.0,6.934975e+06,5.978636e+06,2.829920e+06,1.380448e+06,1.361706e+06,1.141647e+06,986478.000000,939934.000000,727505.000000,681888.000000,429031.000000,216749.000000,75388.000000,26162.000000,21931.000000
1,1.314980e+00,1.032159e+00,0.032159,1.0,2.922155e-01,2.519187e-01,1.192429e-01,5.816724e-02,5.737751e-02,4.810500e-02,0.041567,0.039606,0.030655,0.028732,0.018078,0.009133,0.003177,0.001102,0.000924


In [4]:
elections_1953_df.columns
col = elections_1953_df[elections_1953_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53',
       'Unnamed: 54', 'Unnamed: 55'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['DP.1']

[[33120940.0 28479550.0 '959790' '928278' '27519760' '27551272' '9577659'
  '10016594' '8131257' '7944943' '2967566' '2629163' '2450286' '2427387'
  '1613215' '1616953' '1073031' '896128' '611317' '607860' '399070'
  '465641' '286465' '318475' '204725' '295739' '55835' '217078' '78356'
  '70726' '44339' '44585' '6269' '2531' '654' '17185']]
['27519760']


0       896128
1    0.0325631
Name: DP.1, dtype: object

In [5]:
elections_1957_df.columns
col = elections_1957_df[elections_1957_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['GB/BHE.1']

[[35400923.0 31072894.0 '916680' '1167466' '30156214' '29905428'
  '11975400' '11875339' '9651669' '9495571' '3186150' '3133060' '2276234'
  '2307135' '1324636' '1374066' '1062293' '1007282' '290622' '308564'
  '295533' '254322' '37329' '58725' '3024' '36592' '16410' '17490'
  '33463' '32262' '2250' '5020' '356' '845']]
['30156214']


0      1374066
1    0.0455649
Name: GB/BHE.1, dtype: object

In [6]:
print(elections_1961_df.columns)
#"""
col = elections_1961_df[elections_1961_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['FDP']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'SPD', 'SPD.1',
       'CDU', 'CDU.1', 'CSU', 'CSU.1', 'FDP', 'FDP.1', 'GDP', 'GDP.1', 'DFU',
       'DFU.1', 'DRP', 'DRP.1', 'DG', 'DG.1', 'SSW', 'SSW.1', 'WGnD', 'Übrige',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45'],
      dtype='object')
[[37440715.0 32849624.0 '845158' '1298723' '32004466' '31550901'
  '11672057' '11427355' '11622995' '11283901' '3104742' '3014471'
  '3866269' '4028766' '859290' '870756' '587488' '609918' '242649'
  '262977' '21083' '27308' '24951' '25449' '778' '2164']]
['32004466']


0     3866269
1    0.120804
Name: FDP, dtype: object

In [7]:
print(elections_1965_df.columns)
#"""
col = elections_1965_df[elections_1965_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['NPD']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'SPD', 'SPD.1',
       'CDU', 'CDU.1', 'CSU', 'CSU.1', 'FDP', 'FDP.1', 'NPD', 'NPD.1', 'DFU',
       'DFU.1', 'AUD', 'AUD.1', 'CVP', 'CVP.1', 'FSU', 'FSU.1', 'UAP', 'UAP.1',
       'EFP', 'Übrige', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47'],
      dtype='object')
[[38510395.0 33416207.0 '979158' '795765' '32437049' '32620442'
  '12998474' '12813186' '12631319' '12387562' '3204648' '3136506'
  '2562294' '3096739' '587216' '664193' '386900' '434182' '46146' '52637'
  '11978' '19832' '6287' '10631' '1127' '3959' '1015' '660']]
['32437049']


0       587216
1    0.0181032
Name: NPD, dtype: object

In [8]:
print(elections_1969_df.columns)
#"""
col = elections_1969_df[elections_1969_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['CSU']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'SPD', 'SPD.1',
       'CDU', 'CDU.1', 'CSU', 'CSU.1', 'FDP', 'FDP.1', 'NPD', 'NPD.1', 'ADF',
       'ADF.1', 'BP', 'BP.1', 'EP', 'EP.1', 'GPD', 'FSU', 'FSU.1', 'ZENTRUM',
       'UAP', 'UAP.1', 'DV', 'Übrige', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49'],
      dtype='object')
[[38677235.0 33523064.0 '809548' '557040' '32713516' '32966024'
  '14402374' '14065716' '12137148' '12079535' '3094176' '3115652'
  '1554651' '1903422' '1189375' '1422010' '209180' '197331' '54940'
  '49694' '20927' '49650' '45401' '10192' '16371' '15933' '1531' '5309'
  '461' '38561']]
['32713516']


0     3094176
1    0.094584
Name: CSU, dtype: object

In [9]:
print(elections_1972_df.columns)
#"""
col = elections_1972_df[elections_1972_df['Unnamed: 1'] == 'Bundesgebiet ohne Berlin']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['F.D.P.']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'SPD', 'SPD.1',
       'CDU', 'CDU.1', 'CSU', 'CSU.1', 'F.D.P.', 'F.D.P..1', 'DKP', 'DKP.1',
       'EFP', 'EFP.1', 'FSU', 'FSU.1', 'NPD', 'NPD.1', 'Übrige', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42'],
      dtype='object')
[[41446302.0 37761589.0 '457810' '301839' '37303779' '37459750'
  '18228239' '17175169' '13304813' '13190837' '3620625' '3615183'
  '1790513' '3129982' '146258' '113891' '7581' '24057' '1864' '3166'
  '194389' '207465' '9497']]
['37303779']


0      1790513
1    0.0479982
Name: F.D.P., dtype: object

In [10]:
print(elections_1976_df.columns)
#"""
col = elections_1976_df[elections_1976_df['Unnamed: 1'] == 'BUNDESGEBIET OHNE BERLIN']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['AUD']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'S P D', 'S P D.1',
       'C D U', 'C D U.1', 'C S U', 'C S U.1', 'F.D.P.', 'F.D.P..1', 'AUD',
       'AUD.1', 'AVP', 'AVP.1', 'C.B.V.', 'C.B.V..1', 'D K P', 'D K P.1',
       'E A P', 'E A P.1', '5%-BLOCK', '5%-BLOCK.1', 'GIM', 'GIM.1', 'KPD',
       'KPD.1', 'K B W', 'K B W.1', 'N P D', 'N P D.1', 'RFP', 'U A P',
       'U A P.1', 'VL', 'VL.1', 'Übrige'],
      dtype='object')
[[42058015.0 38165753.0 '470109' '343253' '37695644' '37822500'
  '16471321' '16099019' '14423157' '14367302' '4008514' '4027499'
  '2417683' '2995085' '19490' '22202' '2636' '4723' '4876' '6720'
  '170855' '118581' '3177' '6811' '985' '2940' '2037' '4759' '8822'
  '22714' '21414' '20018' '136028' '122661' '227' '499' '765' '217' '701'
  '3706']]
['37695644']


0          19490
1    0.000517036
Name: AUD, dtype: object

In [11]:
print(elections_1980_df.columns)
#"""
col = elections_1980_df[elections_1980_df['Unnamed: 1'] == 'BUNDESGEBIET OHNE BERLIN']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['BUERGERP']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'S P D', 'S P D.1',
       'C D U', 'C D U.1', 'C S U', 'C S U.1', 'F.D.P.', 'F.D.P..1',
       'BUERGERP', 'BUERGERP.1', 'C.B.V.', 'C.B.V..1', 'D K P', 'D K P.1',
       'GRUENE', 'GRUENE.1', 'E A P', 'E A P.1', 'K B W', 'K B W.1', 'N P D',
       'N P D.1', 'V', 'V.1', 'D F P', 'D U', 'U A P', 'UEBRIGE'],
      dtype='object')
[[43231741.0 38292176.0 '485645' '353115' '37806531' '37938981'
  '16808861' '16260677' '13467207' '12989200' '3941365' '3908459'
  '2720480' '4030999' '507' '11256' '0' '3946' '107158' '71600' '732619'
  '569589' '4992' '7666' '12008' '8174' '0' '68096' '7160' '9319' '96'
  '421' '159' '3498']]
['37806531']


0            507
1    1.34104e-05
Name: BUERGERP, dtype: object

In [12]:
print(elections_1983_df.columns)
#"""
col = elections_1983_df[elections_1983_df['Unnamed: 1'] == 'BUNDESGEBIET OHNE BERLIN']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'S P D', 'S P D.1',
       'C D U', 'C D U.1', 'C S U', 'C S U.1', 'F.D.P.', 'F.D.P..1', 'GRUENE',
       'GRUENE.1', 'B W K', 'B W K.1', 'C.B.V.', 'C.B.V..1', 'D K P',
       'D K P.1', 'E A P', 'E A P.1', 'K P D', 'K P D.1', 'N P D', 'N P D.1',
       'OE D P', 'OE D P.1', 'U S D', 'U S D.1', 'UEBRIGE'],
      dtype='object')
[[44088935.0 39279529.0 '434176' '338841' '38845353' '38940687'
  '15686033' '14865807' '15943460' '14857680' '4318800' '4140865'
  '1087918' '2706942' '1609855' '2167431' '686' '2129' '2068' '10994'
  '96143' '64986' '7491' '14966' '0' '3431' '57112' '91095' '3341'
  '11028' '450' '3333' '31996']]
['38845353']


,Wahlberechtigte,Wähler,Ungültige,Ungültige.1,Gültige,Gültige.1,S P D,S P D.1,C D U,C D U.1,C S U,C S U.1,F.D.P.,F.D.P..1,GRUENE,GRUENE.1,B W K,B W K.1,C.B.V.,C.B.V..1,D K P,D K P.1,E A P,E A P.1,K P D,K P D.1,N P D,N P D.1,OE D P,OE D P.1,U S D,U S D.1,UEBRIGE
0,4.408894e+07,3.927953e+07,434176,338841,38845353,38940687,15686033,14865807,15943460,14857680,4318800,4140865,1087918,2706942,1609855,2167431,686,2129,2068,10994,96143,64986,7491,14966,0,3431,57112,91095,3341,11028,450,3333,31996
1,1.134986e+00,1.011177e+00,0.011177,0.00872282,1,1.00245,0.403807,0.382692,0.410434,0.382483,0.111179,0.106599,0.0280064,0.0696851,0.0414427,0.0557964,1.76598e-05,5.48071e-05,5.32367e-05,0.00028302,0.00247502,0.00167294,0.000192842,0.000385271,0,8.83246e-05,0.00147024,0.00234507,8.60077e-05,0.000283895,1.15844e-05,8.58018e-05,0.000823676


In [13]:
print(elections_1987_df.columns)
#"""
col = elections_1987_df[elections_1987_df['Unnamed: 1'] == 'BUNDESGEBIET OHNE BERLIN']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['F.D.P..1']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'S P D', 'S P D.1',
       'C D U', 'C D U.1', 'C S U', 'C S U.1', 'F.D.P.', 'F.D.P..1', 'GRUENE',
       'GRUENE.1', 'A S D', 'A S D.1', 'B P', 'B P.1', 'C.B.V.', 'C.B.V..1',
       'ZENTRUM', 'ZENTRUM.1', 'MUENDIGE', 'MUENDIGE.1', 'FRAUEN', 'FRAUEN.1',
       'F A P', 'F A P.1', 'M L P D', 'M L P D.1', 'N P D', 'N P D.1',
       'OE D P', 'OE D P.1', 'PATRIOTEN', 'PATRIOTEN.1', 'FAMILIE',
       'SOLIDARITAET', 'F S U', 'H P', 'U A P', 'UEBRIGE', 'Unnamed: 47',
       'Unnamed: 48'],
      dtype='object')
[[45327982.0 38225294.0 '482481' '357975' '37742813' '37867319'
  '14787953' '14025763' '14168527' '13045745' '3859244' '3715827'
  '1760496' '3440911' '2649459' '3126256' '3151' '1834' '8024' '26367'
  '741' '5282' '4020' '19035' '611' '24630' '0' '62904' '349' '405' '596'
  '13422' '182880' '227054' '40765' '109152' '27352' '22732' '130' '754'
  '110' '

0      3440911
1    0.0911673
Name: F.D.P..1, dtype: object

In [14]:
print(elections_1990_df.columns)
#"""
col = elections_1990_df[elections_1990_df['Unnamed: 1'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['PDS.1']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'CDU', 'CDU.1',
       'CSU', 'CSU.1', 'SPD', 'SPD.1', 'FDP', 'FDP.1', 'GRÜNE', 'GRÜNE.1',
       'PDS', 'PDS.1', 'DSU', 'DSU.1', 'B90/Gr', 'B90/Gr.1', 'BP', 'BP.1',
       'DDD', 'DDD.1', 'BSA', 'BSA.1', 'LIGA', 'LIGA.1', 'CM', 'CM.1',
       'ÖKO-Union', 'ÖKO-Union.1', 'DIE GRAUEN', 'DIE GRAUEN.1', 'Mündige',
       'Mündige.1', 'REP', 'REP.1', 'Frauen', 'Frauen.1', 'KPD', 'KPD.1',
       'NPD', 'NPD.1', 'ÖDP', 'ÖDP.1', 'Patrioten', 'Patrioten.1', 'SpAD',
       'SpAD.1', 'VAA', 'VAA.1', 'EFP', 'Übrige'],
      dtype='object')
[[60436560.0 46995915.0 '720990' '540143' '46274925' '46455772'
  '17707574' '17055116' '3423904' '3302980' '16279980' '15545366'
  '3595135' '5123233' '2037885' '1788200' '1049245' '1129578' '131747'
  '89008' '552027' '559207' '10836' '31315' '474' '1009' '214' '826'
  '8667' '39640' '9824' '36446' '1106' '4661' '218412' '385910' '

0      1129578
1    0.0244102
Name: PDS.1, dtype: object

In [15]:
print(elections_1994_df.columns)
#"""
col = elections_1994_df[elections_1994_df['Unnamed: 1'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['PDS']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'CDU', 'CDU.1',
       'SPD', 'SPD.1', 'FDP', 'FDP.1', 'CSU', 'CSU.1', 'GRÜNE', 'GRÜNE.1',
       'PDS', 'PDS.1', 'REP', 'REP.1', 'APD', 'APD.1', 'BP', 'BP.1', 'BüSo',
       'BüSo.1', 'BSA', 'BSA.1', 'LIGA', 'LIGA.1', 'CM', 'CM.1', 'ZENTRUM',
       'ZENTRUM.1', 'GRAUE', 'GRAUE.1', 'NATURGESETZ', 'NATURGESETZ.1', 'MLPD',
       'MLPD.1', 'Tierschutz', 'Tierschutz.1', 'ödp', 'ödp.1', 'PBC', 'PBC.1',
       'PASS', 'PASS.1', 'STATT Partei', 'STATT Partei.1', 'BGD', 'DKP', 'DSU',
       'DVP', 'DEMOKRATEN', 'FBU', 'FSU', 'KPD', 'LD', 'UAP',
       'Einzelbewerber'],
      dtype='object')
[[60452009.0 47737999.0 '788643' '632825' '46949356' '47105174'
  '17473325' '16089960' '17966813' '17140354' '1558185' '3258407'
  '3657627' '3427196' '3037902' '3424315' '1920420' '2066176' '787757'
  '875239' '1654' '21533' '3324' '42491' '8032' '8103' '0' '1285' '3788'
  '

0      1920420
1    0.0409041
Name: PDS, dtype: object

In [16]:
print(elections_1998_df.columns)
#"""
col = elections_1998_df[elections_1998_df['Unnamed: 1'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['PDS.1']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'SPD', 'SPD.1',
       'CDU', 'CDU.1', 'CSU', 'CSU.1', 'GRÜNE', 'GRÜNE.1', 'F.D.P.',
       'F.D.P..1', 'PDS', 'PDS.1', 'Deutschland', 'Deutschland.1', 'APPD',
       'APPD.1', 'APD', 'APD.1', 'BP', 'BP.1', 'BüSo', 'BüSo.1', 'BFB',
       'BFB.1', 'CHANCE 2000', 'CHANCE 2000.1', 'CM', 'CM.1', 'DPD', 'DPD.1',
       'DVU', 'GRAUE', 'GRAUE.1', 'REP', 'REP.1', 'FAMILIE', 'FAMILIE.1',
       'DIE FRAUEN', 'DIE FRAUEN.1', 'HP', 'HP.1', 'Pro DM', 'MLPD', 'MLPD.1',
       'Tierschutz', 'Tierschutz.1', 'NPD', 'NPD.1', 'NATURGESETZ',
       'NATURGESETZ.1', 'FORUM', 'FORUM.1', 'ödp', 'ödp.1', 'PBC', 'PBC.1',
       'AB 2000', 'AB 2000.1', 'Nichtwähler', 'PASS', 'PASS.1', 'PSG', 'DKP',
       'DSU', 'ZENTRUM', 'DMP', 'FP Deutschl.', 'FSU', 'STATT Partei',
       'Übrige'],
      dtype='object')
[[60762751.0 49947087.0 '780507' '638575' '49166580' '49308512'
  '2153589

0      2515454
1    0.0511619
Name: PDS.1, dtype: object

In [17]:
print(elections_2002_df.columns)
#"""
col = elections_2002_df[elections_2002_df['Unnamed: 1'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Wahlkreis', 'Unnamed: 1', 'Land'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['PDS']
#"""

Index(['Wahlkreis', 'Unnamed: 1', 'Land', 'Wahlberechtigte', 'Wähler',
       'Ungültige', 'Ungültige.1', 'Gültige', 'Gültige.1', 'SPD', 'SPD.1',
       'CDU', 'CDU.1', 'CSU', 'CSU.1', 'GRÜNE', 'GRÜNE.1', 'FDP', 'FDP.1',
       'PDS', 'PDS.1', 'REP', 'REP.1', 'GRAUE', 'GRAUE.1', 'Tierschutz',
       'Tierschutz.1', 'NPD', 'NPD.1', 'ödp', 'ödp.1', 'PBC', 'PBC.1',
       'DIE FRAUEN', 'DIE FRAUEN.1', 'BP', 'BP.1', 'FAMILIE', 'FAMILIE.1',
       'CM', 'CM.1', 'BüSo', 'BüSo.1', 'HP', 'HP.1', 'Violetten',
       'Violetten.1', 'AUFBRUCH', 'AUFBRUCH.1', 'ZENTRUM', 'ZENTRUM.1', 'KPD',
       'KPD.1', 'PRG', 'PRG.1', 'Schill', 'Schill.1', 'Deutschland', 'DKP',
       'DSU', 'FP Deutschl.', 'Übrige'],
      dtype='object')
[[61432868.0 48582761.0 '741037' '586281' '47841724' '47996480'
  '20059967' '18488668' '15336512' '14167561' '4311178' '4315080'
  '2693794' '4110355' '2752796' '3538815' '2079203' '1916702' '55947'
  '280671' '75490' '114224' '8858' '159655' '103209' '215232' '56593'
  '568

0    2079203
1    0.04346
Name: PDS, dtype: object

In [18]:
print(elections_2005_df.columns)
#"""
col = elections_2005_df[elections_2005_df['Gebiet'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Gebiet', 'Unnamed: 140', 'Unnamed: 141',
       'Unnamed: 142', 'Übrige', 'Unnamed: 144', 'Unnamed: 145',
       'Unnamed: 146', 'Unnamed: 147'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['Die Linke.']
#"""

Index(['Nr', 'Gebiet', 'gehört', 'Wahlberechtigte', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Wähler', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 138', 'UNABHÄNGIGE', 'Unnamed: 140', 'Unnamed: 141',
       'Unnamed: 142', 'Übrige', 'Unnamed: 144', 'Unnamed: 145',
       'Unnamed: 146', 'Unnamed: 147'],
      dtype='object', length=148)
[[999.0 0 '61870711' '61432868' 0 0 '48044134' '48582761' 0 0 '850072'
  '741037' '756146' '586281' '47194062' '47841724' '47287988' '47996480'
  '18129100' '20059967' '16194665' '18488668' '15390950' '15336512'
  '13136740' '14167561' '3889990' '4311178' '3494309' '4315080' '2538913'
  '2693794' '3838326' '4110355' '2208531' '2752796' '4648144' '3538815'
  '3764168' '2079203' '4118194' '1916702' '5401' '120330' '3338' '400476'
  '38678' '55947' '266101' '280671' '857777' '103209' '748568' '215232'
  '7341' '8858' '110603' '159655' '6340' '75490' '198601' '114224'
  '57027' '71106' '108605' '101645' '1327' '2264' '27497' '36832' '760

0      3764168
1    0.0797594
Name: Die Linke., dtype: object

In [19]:
print(elections_2009_df.columns)
#"""
col = elections_2009_df[elections_2009_df['Gebiet'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Gebiet', 'Unnamed: 132', 'Unnamed: 133',
       'Unnamed: 134', 'Unnamed: 135'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['NPD']
#"""

Index(['Nr', 'Gebiet', 'gehört', 'Wahlberechtigte', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Wähler', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 126', 'Freie Union', 'Unnamed: 128', 'Unnamed: 129',
       'Unnamed: 130', 'Übrige', 'Unnamed: 132', 'Unnamed: 133',
       'Unnamed: 134', 'Unnamed: 135'],
      dtype='object', length=136)
[[999.0 0 '62168489' '61870711' 0 0 '44005575' '48044134' 0 0 '757575'
  '850072' '634385' '756146' '43248000' '47194062' '43371190' '47287988'
  '12079758' '18129100' '9990488' '16194665' '13856674' '15390950'
  '11828277' '13136740' '4076496' '2208531' '6316080' '4648144' '4791124'
  '3764168' '5155933' '4118194' '3977125' '2538913' '4643272' '3838326'
  '3191000' '3889990' '2830238' '3494309' '768442' '857777' '635525'
  '748568' '30061' '38678' '193396' '266101' '17848' '76064' '120718'
  '191842' '16887' '7341' '230872' '110603' '12052' '57027' '40370'
  '108605' '17512' '16480' '29261' '45238' '34894' '40984' '38706'
  '35649' 

0       768442
1    0.0177683
Name: NPD, dtype: object

In [20]:
print(elections_2013_df.columns)
#"""
col = elections_2013_df[elections_2013_df['Gebiet'] == 'Bundesgebiet']
col = col.fillna(0)
valid_votes = col['Gültige'].values
col1 = col.drop(['Gebiet', 'Unnamed: 152', 'Unnamed: 153', 'Unnamed: 154',
       'Übrige', 'Unnamed: 156', 'Unnamed: 157', 'Unnamed: 158',
       'Unnamed: 159'], axis=1)

print(col1.values)
print(valid_votes)
perc = col1.values.astype(int)/int(valid_votes)
col1 = col1.append(pd.DataFrame(perc, columns=col1.columns), ignore_index=True)
col1['AfD']
#"""

Index(['Nr', 'Gebiet', 'gehört', 'Wahlberechtigte', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Wähler', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 150', 'NEIN!', 'Unnamed: 152', 'Unnamed: 153', 'Unnamed: 154',
       'Übrige', 'Unnamed: 156', 'Unnamed: 157', 'Unnamed: 158',
       'Unnamed: 159'],
      dtype='object', length=160)
[[999.0 0 '61946900' '62168489' 0 0 '44309925' '44005575' 0 0 '684883'
  '757575' '583069' '634385' '43625042' '43248000' '43726856' '43371190'
  '16233642' '13856674' '14921877' '11828277' '12843458' '12079758'
  '11252215' '9990488' '1028645' '4076496' '2083533' '6316080' '3585178'
  '4791124' '3755699' '5155933' '3180299' '3977125' '3694057' '4643272'
  '3544079' '3191000' '3243569' '2830238' '963623' '46770' '959177'
  '847870' '635135' '768442' '560828' '635525' '4437' '16887' '140366'
  '230872' '27299' '30061' '91193' '193396' '128209' '105653' '127088'
  '132249' '4478' '17848' '7449' '120718' '5324' '37946' '8578' '100605'
  '920'

0       810915
1    0.0185883
Name: AfD, dtype: object

## List of parties that had over 5% on a federal level

- 1949: SPD, CDU, FDP, CSU, KPD
- 1953: CDU, SPD, FDP, CSU, GB/BHE
- 1957: CDU, SPD, FDP, CSU
- 1961: CDU, SPD, FDP, CSU
- 1965: SPD, CDU, CSU, FDP
- 1969: SPD, CDU, CSU
- 1972: SPD, CDU, CSU
- 1976: SPD, CDU, CSU, FDP
- 1980: SPD, CDU, CSU, FDP
- 1983: SPD, CDU, CSU, FDP, GRUENE
- 1987: SPD, CDU, CSU, FDP, GRUENE
- 1990: SPD, CDU, CSU, FDP
- 1994: SPD, CDU, CSU, FDP, GRUENE
- 1998: SPD, CDU, CSU, FDP, GRUENE, PDS (=LINKE)
- 2002: SPD, CDU, CSU, FDP, GRUENE
- 2005: SPD, CDU, CSU, FDP, GRUENE, LINKE
- 2009: SPD, CDU, CSU, FDP, GRUENE, LINKE
- 2013: SPD, CDU, CSU, GRUENE, LINKE
- 2017: SPD, CDU, CSU, GRUENE, LINKE, FDP, AFD
- ever: CDU/CSU, SPD, FDP, GRUENE, LINKE, AFD, KPD, GB/BHE

# Data wrangling

All clean and final datasets have to fulfill these requirements:
1. The have exactly the columns ['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler', 'Ungültige', 'Gültige', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev', 'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE', 'LINKE_prev', 'LINKE:2', LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev', 'KPD', 'KPD_prev', 'KPD:2', KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev']
2. All rows have meaning; empty spacing rows are removed

In [21]:
###################### 1949 ###########################

#print(elections_1949_df.columns)
elections_1949_df_clean = elections_1949_df.drop(['Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
elections_1949_df_clean = elections_1949_df_clean.fillna(0)

#rename unnamed: 1
elections_1949_df_clean = elections_1949_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1949_df_clean = elections_1949_df_clean.drop([0, 1, 17, 27, 63, 68, 136, 160, 177, 212])

#create the final df
elections_1949_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Gültige', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1949_df_final['Wahlkreis'] = elections_1949_df_clean['Wahlkreis']
elections_1949_df_final['Gebiet'] = elections_1949_df_clean['Gebiet']
elections_1949_df_final['Wahlberechtigte'] = elections_1949_df_clean['Wahlberechtigte']
elections_1949_df_final['Wähler'] = elections_1949_df_clean['Wähler']
elections_1949_df_final['Ungültige'] = elections_1949_df_clean['Ungültige']
elections_1949_df_final['Gültige'] = elections_1949_df_clean['Gültige']
elections_1949_df_final['CDU/CSU'] = elections_1949_df_clean['CDU'] + elections_1949_df_clean['CSU']
elections_1949_df_final['SPD'] = elections_1949_df_clean['SPD']
elections_1949_df_final['FDP'] = elections_1949_df_clean['FDP']
elections_1949_df_final['KPD'] = elections_1949_df_clean['KPD']
elections_1949_df_final['OTHER'] = elections_1949_df_clean['Parteilose'] + \
                                   elections_1949_df_clean['BP'] + \
                                   elections_1949_df_clean['DP'] + \
                                   elections_1949_df_clean['Zentrum'] +\
                                   elections_1949_df_clean['WAV'] +\
                                   elections_1949_df_clean['DKP/DRP'] +\
                                   elections_1949_df_clean['RSF'] +\
                                   elections_1949_df_clean['SSW'] +\
                                   elections_1949_df_clean['EVD'] +\
                                   elections_1949_df_clean['RWVP'] 
#fill NaNs with 0s
elections_1949_df_final = elections_1949_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1949_df_final['Gültige'] - elections_1949_df_final['CDU/CSU'] - elections_1949_df_final['SPD'] - elections_1949_df_final['FDP'] - elections_1949_df_final['KPD']
B = elections_1949_df_final['OTHER']

assert(np.allclose(A, B))

#save file as csv
elections_1949_df_final.to_csv('Bundestagswahlen_clean/elections_1949.csv')

In [22]:
###################### 1953 ###########################

#print(elections_1953_df.columns)
elections_1953_df_clean = elections_1953_df.drop(['Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53',
       'Unnamed: 54', 'Unnamed: 55'], axis=1)

elections_1953_df_clean = elections_1953_df_clean.fillna(0)

#rename unnamed: 1
elections_1953_df_clean = elections_1953_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1953_df_clean = elections_1953_df_clean.drop([0, 1, 17, 27, 63, 68, 136, 160, 177, 212])

#create the final df
elections_1953_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1953_df_final['Wahlkreis'] = elections_1953_df_clean['Wahlkreis']
elections_1953_df_final['Gebiet'] = elections_1953_df_clean['Gebiet']
elections_1953_df_final['Wahlberechtigte'] = elections_1953_df_clean['Wahlberechtigte'].astype('int64')
elections_1953_df_final['Wähler'] = elections_1953_df_clean['Wähler'].astype('int64')
elections_1953_df_final['Ungültige'] = elections_1953_df_clean['Ungültige'].astype('int64')
elections_1953_df_final['Ungültige:2'] = elections_1953_df_clean['Ungültige.1'].astype('int64')
elections_1953_df_final['Gültige'] = elections_1953_df_clean['Gültige'].astype('int64')
elections_1953_df_final['Gültige:2'] = elections_1953_df_clean['Gültige.1'].astype('int64')
elections_1953_df_final['CDU/CSU'] = elections_1953_df_clean['CDU'].astype('int64') + elections_1953_df_clean['CSU'].astype('int64')
elections_1953_df_final['CDU/CSU:2'] = elections_1953_df_clean['CDU.1'].astype('int64') + elections_1953_df_clean['CSU.1'].astype('int64')
elections_1953_df_final['SPD'] = elections_1953_df_clean['SPD'].astype('int64')
elections_1953_df_final['SPD:2'] = elections_1953_df_clean['SPD.1'].astype('int64')
elections_1953_df_final['FDP'] = elections_1953_df_clean['FDP'].astype('int64')
elections_1953_df_final['FDP:2'] = elections_1953_df_clean['FDP.1'].astype('int64')
elections_1953_df_final['BHE'] = elections_1953_df_clean['GB/BHE'].astype('int64')
elections_1953_df_final['BHE:2'] = elections_1953_df_clean['GB/BHE.1'].astype('int64')
elections_1953_df_final['KPD'] = elections_1953_df_clean['KPD'].astype('int64')
elections_1953_df_final['KPD:2'] = elections_1953_df_clean['KPD.1'].astype('int64')

#others
elections_1953_df_final['OTHER'] = elections_1953_df_clean['DP'].astype('int64') + \
                                   elections_1953_df_clean['BP'].astype('int64') + \
                                   elections_1953_df_clean['GVP'].astype('int64') +\
                                   elections_1953_df_clean['DRP'].astype('int64') +\
                                   elections_1953_df_clean['Zentrum'].astype('int64') +\
                                   elections_1953_df_clean['DNS'].astype('int64') +\
                                   elections_1953_df_clean['SSW'].astype('int64') +\
                                   elections_1953_df_clean['SHLP'].astype('int64') +\
                                   elections_1953_df_clean['VU'].astype('int64') +\
                                   elections_1953_df_clean['PdgD'].astype('int64') +\
                                   elections_1953_df_clean['Übrige'].astype('int64')
#others: Zweitstimmen
elections_1953_df_final['OTHER:2'] = elections_1953_df_clean['DP.1'].astype('int64') + \
                                   elections_1953_df_clean['BP.1'].astype('int64') + \
                                   elections_1953_df_clean['GVP.1'].astype('int64') +\
                                   elections_1953_df_clean['DRP.1'].astype('int64') +\
                                   elections_1953_df_clean['Zentrum.1'].astype('int64') +\
                                   elections_1953_df_clean['DNS.1'].astype('int64') +\
                                   elections_1953_df_clean['SSW.1'].astype('int64')

#fill NaNs with 0s
elections_1953_df_final = elections_1953_df_final.fillna(0)


#double-check that all votes have been counted
A = elections_1953_df_final['Gültige'] - elections_1953_df_final['CDU/CSU'] - elections_1953_df_final['SPD'] \
  - elections_1953_df_final['FDP'] - elections_1953_df_final['BHE'] - elections_1953_df_final['KPD']
B = elections_1953_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1953_df_final['Gültige:2'] - elections_1953_df_final['CDU/CSU:2'] - elections_1953_df_final['SPD:2']\
   - elections_1953_df_final['FDP:2'] - elections_1953_df_final['BHE:2'] - elections_1953_df_final['KPD:2']
B2 = elections_1953_df_final['OTHER:2']
assert(np.allclose(A2, B2))

#add the results of previous years
#TODO

#save file as csv
elections_1953_df_final.to_csv('Bundestagswahlen_clean/elections_1953.csv')

In [23]:
###################### 1957 ###########################

elections_1957_df_clean = elections_1957_df.drop(['Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53'], axis=1)

elections_1957_df_clean = elections_1957_df_clean.fillna(0)

#rename unnamed: 1
elections_1957_df_clean = elections_1957_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1957_df_clean = elections_1957_df_clean.drop([0, 1, 17, 27, 63, 68, 136, 160, 177, 212, 261])

#create the final df
elections_1957_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1957_df_final['Wahlkreis'] = elections_1957_df_clean['Wahlkreis']
elections_1957_df_final['Gebiet'] = elections_1957_df_clean['Gebiet']
elections_1957_df_final['Wahlberechtigte'] = elections_1957_df_clean['Wahlberechtigte'].astype('int64')
elections_1957_df_final['Wähler'] = elections_1957_df_clean['Wähler'].astype('int64')
elections_1957_df_final['Ungültige'] = elections_1957_df_clean['Ungültige'].astype('int64')
elections_1957_df_final['Ungültige:2'] = elections_1957_df_clean['Ungültige.1'].astype('int64')
elections_1957_df_final['Gültige'] = elections_1957_df_clean['Gültige'].astype('int64')
elections_1957_df_final['Gültige:2'] = elections_1957_df_clean['Gültige.1'].astype('int64')
elections_1957_df_final['CDU/CSU'] = elections_1957_df_clean['CDU'].astype('int64') + elections_1957_df_clean['CSU'].astype('int64')
elections_1957_df_final['CDU/CSU:2'] = elections_1957_df_clean['CDU.1'].astype('int64') + elections_1957_df_clean['CSU.1'].astype('int64')
elections_1957_df_final['SPD'] = elections_1957_df_clean['SPD'].astype('int64')
elections_1957_df_final['SPD:2'] = elections_1957_df_clean['SPD.1'].astype('int64')
elections_1957_df_final['FDP'] = elections_1957_df_clean['FDP'].astype('int64')
elections_1957_df_final['FDP:2'] = elections_1957_df_clean['FDP.1'].astype('int64')
elections_1957_df_final['BHE'] = elections_1957_df_clean['GB/BHE'].astype('int64')
elections_1957_df_final['BHE:2'] = elections_1957_df_clean['GB/BHE.1'].astype('int64')

#others
elections_1957_df_final['OTHER'] = elections_1957_df_clean['DP'].astype('int64') + \
                                   elections_1957_df_clean['DRP'].astype('int64') + \
                                   elections_1957_df_clean['FU'].astype('int64') +\
                                   elections_1957_df_clean['BdD'].astype('int64') +\
                                   elections_1957_df_clean['Mittelstand'].astype('int64') +\
                                   elections_1957_df_clean['DG'].astype('int64') +\
                                   elections_1957_df_clean['SSW'].astype('int64') +\
                                   elections_1957_df_clean['VU'].astype('int64') +\
                                   elections_1957_df_clean['PdgD'].astype('int64') +\
                                   elections_1957_df_clean['Übrige'].astype('int64')

#others: Zweitstimmen
elections_1957_df_final['OTHER:2'] = elections_1957_df_clean['DP.1'].astype('int64') + \
                                   elections_1957_df_clean['DRP.1'].astype('int64') + \
                                   elections_1957_df_clean['FU.1'].astype('int64') +\
                                   elections_1957_df_clean['BdD.1'].astype('int64') +\
                                   elections_1957_df_clean['Mittelstand.1'].astype('int64') +\
                                   elections_1957_df_clean['DG.1'].astype('int64') +\
                                   elections_1957_df_clean['SSW.1'].astype('int64') +\
                                   elections_1957_df_clean['VU.1'].astype('int64')

#fill NaNs with 0s
elections_1957_df_final = elections_1957_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1957_df_final['Gültige'] - elections_1957_df_final['CDU/CSU'] - elections_1957_df_final['SPD']\
  - elections_1957_df_final['FDP'] - elections_1957_df_final['BHE']
B = elections_1957_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1957_df_final['Gültige:2'] - elections_1957_df_final['CDU/CSU:2'] - elections_1957_df_final['SPD:2']\
  - elections_1957_df_final['FDP:2'] - elections_1957_df_final['BHE:2']
B2 = elections_1957_df_final['OTHER:2']
assert(np.allclose(A2, B2))

#add the results of previous years
#TODO

elections_1957_df_final.to_csv('Bundestagswahlen_clean/elections_1957.csv')

In [24]:
###################### 1961 ###########################

elections_1961_df_clean = elections_1961_df.drop(['Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45'], axis=1)

elections_1961_df_clean = elections_1961_df_clean.fillna(0)

#rename unnamed: 1
elections_1961_df_clean = elections_1961_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1961_df_clean = elections_1961_df_clean.drop([0, 1, 17, 27, 63, 68, 136, 160, 177, 212, 261])

#create the final df
elections_1961_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1961_df_final['Wahlkreis'] = elections_1961_df_clean['Wahlkreis']
elections_1961_df_final['Gebiet'] = elections_1961_df_clean['Gebiet']
elections_1961_df_final['Wahlberechtigte'] = elections_1961_df_clean['Wahlberechtigte'].astype('int64')
elections_1961_df_final['Wähler'] = elections_1961_df_clean['Wähler'].astype('int64')
elections_1961_df_final['Ungültige'] = elections_1961_df_clean['Ungültige'].astype('int64')
elections_1961_df_final['Ungültige:2'] = elections_1961_df_clean['Ungültige.1'].astype('int64')
elections_1961_df_final['Gültige'] = elections_1961_df_clean['Gültige'].astype('int64')
elections_1961_df_final['Gültige:2'] = elections_1961_df_clean['Gültige.1'].astype('int64')
elections_1961_df_final['CDU/CSU'] = elections_1961_df_clean['CDU'].astype('int64') + elections_1961_df_clean['CSU'].astype('int64')
elections_1961_df_final['CDU/CSU:2'] = elections_1961_df_clean['CDU.1'].astype('int64') + elections_1961_df_clean['CSU.1'].astype('int64')
elections_1961_df_final['SPD'] = elections_1961_df_clean['SPD'].astype('int64')
elections_1961_df_final['SPD:2'] = elections_1961_df_clean['SPD.1'].astype('int64')
elections_1961_df_final['FDP'] = elections_1961_df_clean['FDP'].astype('int64')
elections_1961_df_final['FDP:2'] = elections_1961_df_clean['FDP.1'].astype('int64')

#others
elections_1961_df_final['OTHER'] = elections_1961_df_clean['GDP'].astype('int64') + \
                                   elections_1961_df_clean['DFU'].astype('int64') + \
                                   elections_1961_df_clean['DRP'].astype('int64') +\
                                   elections_1961_df_clean['DG'].astype('int64') +\
                                   elections_1961_df_clean['SSW'].astype('int64') +\
                                   elections_1961_df_clean['WGnD'].astype('int64') +\
                                   elections_1961_df_clean['Übrige'].astype('int64')

#others: Zweitstimme
elections_1961_df_final['OTHER:2'] = elections_1961_df_clean['GDP.1'].astype('int64') + \
                                   elections_1961_df_clean['DFU.1'].astype('int64') + \
                                   elections_1961_df_clean['DRP.1'].astype('int64') +\
                                   elections_1961_df_clean['DG.1'].astype('int64') +\
                                   elections_1961_df_clean['SSW.1'].astype('int64')

#fill NaNs with 0s
elections_1961_df_final = elections_1961_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1961_df_final['Gültige'] - elections_1961_df_final['CDU/CSU'] - elections_1961_df_final['SPD']\
- elections_1961_df_final['FDP'] 
B = elections_1961_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1961_df_final['Gültige:2'] - elections_1961_df_final['CDU/CSU:2'] - elections_1961_df_final['SPD:2']\
- elections_1961_df_final['FDP:2'] 
B2 = elections_1961_df_final['OTHER:2']
assert(np.allclose(A2, B2))

#add the results of previous years
#TODO

elections_1961_df_final.to_csv('Bundestagswahlen_clean/elections_1961.csv')

In [25]:
###################### 1965 ###########################

elections_1965_df_clean = elections_1965_df.drop(['Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47'], axis=1)

elections_1965_df_clean = elections_1965_df_clean.fillna(0)

#rename unnamed: 1
elections_1965_df_clean = elections_1965_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1965_df_clean = elections_1965_df_clean.drop([0, 1, 14, 24, 56, 61, 136, 160, 178, 216, 262, 269])

#create the final df
elections_1965_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1965_df_final['Wahlkreis'] = elections_1965_df_clean['Wahlkreis']
elections_1965_df_final['Gebiet'] = elections_1965_df_clean['Gebiet']
elections_1965_df_final['Wahlberechtigte'] = elections_1965_df_clean['Wahlberechtigte'].astype('int64')
elections_1965_df_final['Wähler'] = elections_1965_df_clean['Wähler'].astype('int64')
elections_1965_df_final['Ungültige'] = elections_1965_df_clean['Ungültige'].astype('int64')
elections_1965_df_final['Ungültige:2'] = elections_1965_df_clean['Ungültige.1'].astype('int64')
elections_1965_df_final['Gültige'] = elections_1965_df_clean['Gültige'].astype('int64')
elections_1965_df_final['Gültige:2'] = elections_1965_df_clean['Gültige.1'].astype('int64')
elections_1965_df_final['CDU/CSU'] = elections_1965_df_clean['CDU'].astype('int64') + elections_1965_df_clean['CSU'].astype('int64')
elections_1965_df_final['CDU/CSU:2'] = elections_1965_df_clean['CDU.1'].astype('int64') + elections_1965_df_clean['CSU.1'].astype('int64')
elections_1965_df_final['SPD'] = elections_1965_df_clean['SPD'].astype('int64')
elections_1965_df_final['SPD:2'] = elections_1965_df_clean['SPD.1'].astype('int64')
elections_1965_df_final['FDP'] = elections_1965_df_clean['FDP'].astype('int64')
elections_1965_df_final['FDP:2'] = elections_1965_df_clean['FDP.1'].astype('int64')

#others
elections_1965_df_final['OTHER'] = elections_1965_df_clean['NPD'].astype('int64') + \
                                   elections_1965_df_clean['DFU'].astype('int64') + \
                                   elections_1965_df_clean['AUD'].astype('int64') +\
                                   elections_1965_df_clean['CVP'].astype('int64') +\
                                   elections_1965_df_clean['FSU'].astype('int64') +\
                                   elections_1965_df_clean['UAP'].astype('int64') +\
                                   elections_1965_df_clean['Übrige'].astype('int64')

#others: Zweitstimme
elections_1965_df_final['OTHER:2'] = elections_1965_df_clean['NPD.1'].astype('int64') + \
                                     elections_1965_df_clean['DFU.1'].astype('int64') + \
                                     elections_1965_df_clean['AUD.1'].astype('int64') +\
                                     elections_1965_df_clean['CVP.1'].astype('int64') +\
                                     elections_1965_df_clean['FSU.1'].astype('int64') +\
                                     elections_1965_df_clean['EFP'].astype('int64') +\
                                     elections_1965_df_clean['UAP.1'].astype('int64')

#fill NaNs with 0s
elections_1965_df_final = elections_1965_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1965_df_final['Gültige'] - elections_1965_df_final['CDU/CSU'] - elections_1965_df_final['SPD'] \
    - elections_1965_df_final['FDP'] 
B = elections_1965_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1965_df_final['Gültige:2'] - elections_1965_df_final['CDU/CSU:2'] - elections_1965_df_final['SPD:2']\
    - elections_1965_df_final['FDP:2'] 
B2 = elections_1965_df_final['OTHER:2']
assert(np.allclose(A2, B2))

#add the results of previous years
#TODO

elections_1965_df_final.to_csv('Bundestagswahlen_clean/elections_1965.csv')

In [26]:
###################### 1969 ###########################

elections_1969_df_clean = elections_1969_df.drop(['Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49'], axis=1)

elections_1969_df_clean = elections_1969_df_clean.fillna(0)

#rename unnamed: 1
elections_1969_df_clean = elections_1969_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1969_df_clean = elections_1969_df_clean.drop([0, 1, 14, 24, 56, 61, 136, 160, 178, 216, 262])

#create the final df
elections_1969_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1969_df_final['Wahlkreis'] = elections_1969_df_clean['Wahlkreis']
elections_1969_df_final['Gebiet'] = elections_1969_df_clean['Gebiet']
elections_1969_df_final['Wahlberechtigte'] = elections_1969_df_clean['Wahlberechtigte'].astype('int64')
elections_1969_df_final['Wähler'] = elections_1969_df_clean['Wähler'].astype('int64')
elections_1969_df_final['Ungültige'] = elections_1969_df_clean['Ungültige'].astype('int64')
elections_1969_df_final['Ungültige:2'] = elections_1969_df_clean['Ungültige.1'].astype('int64')
elections_1969_df_final['Gültige'] = elections_1969_df_clean['Gültige'].astype('int64')
elections_1969_df_final['Gültige:2'] = elections_1969_df_clean['Gültige.1'].astype('int64')
elections_1969_df_final['CDU/CSU'] = elections_1969_df_clean['CDU'].astype('int64') + elections_1969_df_clean['CSU'].astype('int64')
elections_1969_df_final['CDU/CSU:2'] = elections_1969_df_clean['CDU.1'].astype('int64') + elections_1969_df_clean['CSU.1'].astype('int64')
elections_1969_df_final['SPD'] = elections_1969_df_clean['SPD'].astype('int64')
elections_1969_df_final['SPD:2'] = elections_1969_df_clean['SPD.1'].astype('int64')
elections_1969_df_final['FDP'] = elections_1969_df_clean['FDP'].astype('int64')
elections_1969_df_final['FDP:2'] = elections_1969_df_clean['FDP.1'].astype('int64')

#others
elections_1969_df_final['OTHER'] = elections_1969_df_clean['NPD'].astype('int64') + \
                                   elections_1969_df_clean['ADF'].astype('int64') + \
                                   elections_1969_df_clean['BP'].astype('int64') +\
                                   elections_1969_df_clean['EP'].astype('int64') +\
                                   elections_1969_df_clean['FSU'].astype('int64') +\
                                   elections_1969_df_clean['UAP'].astype('int64') + \
                                   elections_1969_df_clean['DV'].astype('int64') +\
                                   elections_1969_df_clean['Übrige'].astype('int64')

#others: Zweitstimme
elections_1969_df_final['OTHER:2'] = elections_1969_df_clean['NPD.1'].astype('int64') + \
                                     elections_1969_df_clean['ADF.1'].astype('int64') + \
                                     elections_1969_df_clean['BP.1'].astype('int64') +\
                                     elections_1969_df_clean['EP.1'].astype('int64') +\
                                     elections_1969_df_clean['FSU.1'].astype('int64') +\
                                     elections_1969_df_clean['ZENTRUM'].astype('int64') +\
                                     elections_1969_df_clean['UAP.1'].astype('int64') +\
                                     elections_1969_df_clean['GPD'].astype('int64') 

#fill NaNs with 0s
elections_1969_df_final = elections_1969_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1969_df_final['Gültige'] - elections_1969_df_final['CDU/CSU'] - elections_1969_df_final['SPD'] \
- elections_1969_df_final['FDP'] 
B = elections_1969_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1969_df_final['Gültige:2'] - elections_1969_df_final['CDU/CSU:2'] - elections_1969_df_final['SPD:2'] \
- elections_1969_df_final['FDP:2'] 
B2 = elections_1969_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1969_df_final.to_csv('Bundestagswahlen_clean/elections_1969.csv')

In [29]:
###################### 1972 ###########################

elections_1972_df_clean = elections_1972_df.drop(['Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42'], axis=1)

elections_1972_df_clean = elections_1972_df_clean.fillna(0)

#rename unnamed: 1
elections_1972_df_clean = elections_1972_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1972_df_clean = elections_1972_df_clean.drop([0, 1, 14, 24, 56, 61, 136, 160, 178, 216, 262])

#create the final df
elections_1972_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1972_df_final['Wahlkreis'] = elections_1972_df_clean['Wahlkreis']
elections_1972_df_final['Gebiet'] = elections_1972_df_clean['Gebiet']
elections_1972_df_final['Wahlberechtigte'] = elections_1972_df_clean['Wahlberechtigte'].astype('int64')
elections_1972_df_final['Wähler'] = elections_1972_df_clean['Wähler'].astype('int64')
elections_1972_df_final['Ungültige'] = elections_1972_df_clean['Ungültige'].astype('int64')
elections_1972_df_final['Ungültige:2'] = elections_1972_df_clean['Ungültige.1'].astype('int64')
elections_1972_df_final['Gültige'] = elections_1972_df_clean['Gültige'].astype('int64')
elections_1972_df_final['Gültige:2'] = elections_1972_df_clean['Gültige.1'].astype('int64')
elections_1972_df_final['CDU/CSU'] = elections_1972_df_clean['CDU'].astype('int64') + elections_1972_df_clean['CSU'].astype('int64')
elections_1972_df_final['CDU/CSU:2'] = elections_1972_df_clean['CDU.1'].astype('int64') + elections_1972_df_clean['CSU.1'].astype('int64')
elections_1972_df_final['SPD'] = elections_1972_df_clean['SPD'].astype('int64')
elections_1972_df_final['SPD:2'] = elections_1972_df_clean['SPD.1'].astype('int64')
elections_1972_df_final['FDP'] = elections_1972_df_clean['F.D.P.'].astype('int64')
elections_1972_df_final['FDP:2'] = elections_1972_df_clean['F.D.P..1'].astype('int64')
elections_1972_df_final['KPD'] = elections_1972_df_clean['DKP'].astype('int64')
elections_1972_df_final['KPD:2'] = elections_1972_df_clean['DKP.1'].astype('int64')

#others
elections_1972_df_final['OTHER'] = elections_1972_df_clean['EFP'].astype('int64') + \
                                   elections_1972_df_clean['FSU'].astype('int64') +\
                                   elections_1972_df_clean['NPD'].astype('int64') +\
                                   elections_1972_df_clean['Übrige'].astype('int64')

#others: Zweitstimme
elections_1972_df_final['OTHER:2'] = elections_1972_df_clean['EFP.1'].astype('int64') + \
                                     elections_1972_df_clean['FSU.1'].astype('int64') +\
                                     elections_1972_df_clean['NPD.1'].astype('int64')

#fill NaNs with 0s
elections_1972_df_final = elections_1972_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1972_df_final['Gültige'] - elections_1972_df_final['CDU/CSU'] - elections_1972_df_final['SPD'] \
    - elections_1972_df_final['FDP'] - elections_1972_df_final['KPD']
B = elections_1972_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1972_df_final['Gültige:2'] - elections_1972_df_final['CDU/CSU:2'] - elections_1972_df_final['SPD:2'] \
    - elections_1972_df_final['FDP:2'] - elections_1972_df_final['KPD:2']
B2 = elections_1972_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1972_df_final.to_csv('Bundestagswahlen_clean/elections_1972.csv')

In [30]:
###################### 1976 ###########################

elections_1976_df_clean = elections_1976_df #nothing to drop here

elections_1976_df_clean = elections_1976_df_clean.fillna(0)

#rename unnamed: 1
elections_1976_df_clean = elections_1976_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1976_df_clean = elections_1976_df_clean.drop([0, 1, 14, 24, 56, 61, 136, 160, 178, 216, 262])

#create the final df
elections_1976_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1976_df_final['Wahlkreis'] = elections_1976_df_clean['Wahlkreis']
elections_1976_df_final['Gebiet'] = elections_1976_df_clean['Gebiet']
elections_1976_df_final['Wahlberechtigte'] = elections_1976_df_clean['Wahlberechtigte'].astype('int64')
elections_1976_df_final['Wähler'] = elections_1976_df_clean['Wähler'].astype('int64')
elections_1976_df_final['Ungültige'] = elections_1976_df_clean['Ungültige'].astype('int64')
elections_1976_df_final['Ungültige:2'] = elections_1976_df_clean['Ungültige.1'].astype('int64')
elections_1976_df_final['Gültige'] = elections_1976_df_clean['Gültige'].astype('int64')
elections_1976_df_final['Gültige:2'] = elections_1976_df_clean['Gültige.1'].astype('int64')
elections_1976_df_final['CDU/CSU'] = elections_1976_df_clean['C D U'].astype('int64') + elections_1976_df_clean['C S U'].astype('int64')
elections_1976_df_final['CDU/CSU:2'] = elections_1976_df_clean['C D U.1'].astype('int64') + elections_1976_df_clean['C S U.1'].astype('int64')
elections_1976_df_final['SPD'] = elections_1976_df_clean['S P D'].astype('int64')
elections_1976_df_final['SPD:2'] = elections_1976_df_clean['S P D.1'].astype('int64')
elections_1976_df_final['FDP'] = elections_1976_df_clean['F.D.P.'].astype('int64')
elections_1976_df_final['FDP:2'] = elections_1976_df_clean['F.D.P..1'].astype('int64')
elections_1976_df_final['KPD'] = elections_1976_df_clean['D K P'].astype('int64') + elections_1976_df_clean['KPD'].astype('int64')
elections_1976_df_final['KPD:2'] = elections_1976_df_clean['D K P.1'].astype('int64') + elections_1976_df_clean['KPD.1'].astype('int64')

#others
elections_1976_df_final['OTHER'] = elections_1976_df_clean['AUD'].astype('int64') + \
                                   elections_1976_df_clean['AVP'].astype('int64') + \
                                   elections_1976_df_clean['C.B.V.'].astype('int64') +\
                                   elections_1976_df_clean['E A P'].astype('int64') +\
                                   elections_1976_df_clean['5%-BLOCK'].astype('int64') +\
                                   elections_1976_df_clean['GIM'].astype('int64') +\
                                   elections_1976_df_clean['K B W'].astype('int64') +\
                                   elections_1976_df_clean['N P D'].astype('int64') +\
                                   elections_1976_df_clean['RFP'].astype('int64') +\
                                   elections_1976_df_clean['U A P'].astype('int64') +\
                                   elections_1976_df_clean['VL'].astype('int64') +\
                                   elections_1976_df_clean['Übrige'].astype('int64')

#others: Zweitstimme
elections_1976_df_final['OTHER:2'] = elections_1976_df_clean['AUD.1'].astype('int64') + \
                                   elections_1976_df_clean['AVP.1'].astype('int64') + \
                                   elections_1976_df_clean['C.B.V..1'].astype('int64') +\
                                   elections_1976_df_clean['E A P.1'].astype('int64') +\
                                   elections_1976_df_clean['5%-BLOCK.1'].astype('int64') +\
                                   elections_1976_df_clean['GIM.1'].astype('int64') +\
                                   elections_1976_df_clean['K B W.1'].astype('int64') +\
                                   elections_1976_df_clean['N P D.1'].astype('int64') +\
                                   elections_1976_df_clean['U A P.1'].astype('int64') +\
                                   elections_1976_df_clean['VL.1'].astype('int64')

#fill NaNs with 0s
elections_1976_df_final = elections_1976_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1976_df_final['Gültige'] - elections_1976_df_final['CDU/CSU'] - elections_1976_df_final['SPD'] \
- elections_1976_df_final['FDP'] - elections_1976_df_final['KPD']
B = elections_1976_df_final['OTHER']
assert(np.allclose(A, B))


#double-ckeck that all second votes are counted correctly
A2 = elections_1976_df_final['Gültige:2'] - elections_1976_df_final['CDU/CSU:2'] - elections_1976_df_final['SPD:2'] \
    - elections_1976_df_final['FDP:2'] - elections_1976_df_final['KPD:2'] 
B2 = elections_1976_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1976_df_final.to_csv('Bundestagswahlen_clean/elections_1976.csv')

In [31]:
###################### 1980 ###########################

elections_1980_df_clean = elections_1980_df #nothing to drop here

elections_1980_df_clean = elections_1980_df_clean.fillna(0)

#rename unnamed: 1
elections_1980_df_clean = elections_1980_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1980_df_clean = elections_1980_df_clean.drop([0, 1])

#create the final df
elections_1980_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1980_df_final['Wahlkreis'] = elections_1980_df_clean['Wahlkreis']
elections_1980_df_final['Gebiet'] = elections_1980_df_clean['Gebiet']
elections_1980_df_final['Wahlberechtigte'] = elections_1980_df_clean['Wahlberechtigte'].astype('int64')
elections_1980_df_final['Wähler'] = elections_1980_df_clean['Wähler'].astype('int64')
elections_1980_df_final['Ungültige'] = elections_1980_df_clean['Ungültige'].astype('int64')
elections_1980_df_final['Ungültige:2'] = elections_1980_df_clean['Ungültige.1'].astype('int64')
elections_1980_df_final['Gültige'] = elections_1980_df_clean['Gültige'].astype('int64')
elections_1980_df_final['Gültige:2'] = elections_1980_df_clean['Gültige.1'].astype('int64')
elections_1980_df_final['CDU/CSU'] = elections_1980_df_clean['C D U'].astype('int64') + elections_1980_df_clean['C S U'].astype('int64')
elections_1980_df_final['CDU/CSU:2'] = elections_1980_df_clean['C D U.1'].astype('int64') + elections_1980_df_clean['C S U.1'].astype('int64')
elections_1980_df_final['SPD'] = elections_1980_df_clean['S P D'].astype('int64')
elections_1980_df_final['SPD:2'] = elections_1980_df_clean['S P D.1'].astype('int64')
elections_1980_df_final['FDP'] = elections_1980_df_clean['F.D.P.'].astype('int64')
elections_1980_df_final['FDP:2'] = elections_1980_df_clean['F.D.P..1'].astype('int64')
elections_1980_df_final['GRUENE'] = elections_1980_df_clean['GRUENE'].astype('int64')
elections_1980_df_final['GRUENE:2'] = elections_1980_df_clean['GRUENE.1'].astype('int64')
elections_1980_df_final['KPD'] = elections_1980_df_clean['D K P'].astype('int64')
elections_1980_df_final['KPD:2'] = elections_1980_df_clean['D K P.1'].astype('int64')


#others
elections_1980_df_final['OTHER'] = elections_1980_df_clean['BUERGERP'].astype('int64') + \
                                   elections_1980_df_clean['C.B.V.'].astype('int64') +\
                                   elections_1980_df_clean['E A P'].astype('int64') +\
                                   elections_1980_df_clean['K B W'].astype('int64') +\
                                   elections_1980_df_clean['N P D'].astype('int64') +\
                                   elections_1980_df_clean['V'].astype('int64') +\
                                   elections_1980_df_clean['D F P'].astype('int64') +\
                                   elections_1980_df_clean['D U'].astype('int64') +\
                                   elections_1980_df_clean['U A P'].astype('int64') +\
                                   elections_1980_df_clean['UEBRIGE'].astype('int64')

#others:2 
elections_1980_df_final['OTHER:2'] = elections_1980_df_clean['BUERGERP.1'].astype('int64') + \
                                   elections_1980_df_clean['C.B.V..1'].astype('int64') +\
                                   elections_1980_df_clean['E A P.1'].astype('int64') +\
                                   elections_1980_df_clean['K B W.1'].astype('int64') +\
                                   elections_1980_df_clean['N P D.1'].astype('int64') +\
                                   elections_1980_df_clean['V.1'].astype('int64')
                                   
    
#fill NaNs with 0s
elections_1980_df_final = elections_1980_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1980_df_final['Gültige'] - elections_1980_df_final['CDU/CSU'] - elections_1980_df_final['SPD'] \
- elections_1980_df_final['FDP'] - elections_1980_df_final['GRUENE'] - elections_1980_df_final['KPD']
B = elections_1980_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1980_df_final['Gültige:2'] - elections_1980_df_final['CDU/CSU:2'] - elections_1980_df_final['SPD:2'] \
- elections_1980_df_final['FDP:2'] - elections_1980_df_final['GRUENE:2'] - elections_1980_df_final['KPD:2']
B2 =elections_1980_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1980_df_final.to_csv('Bundestagswahlen_clean/elections_1980.csv')

In [32]:
###################### 1983 ###########################

elections_1983_df_clean = elections_1983_df #nothing to drop here

elections_1983_df_clean = elections_1983_df_clean.fillna(0)

#rename unnamed: 1
elections_1983_df_clean = elections_1983_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1983_df_clean = elections_1983_df_clean.drop([0, 1])

#create the final df
elections_1983_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1983_df_final['Wahlkreis'] = elections_1983_df_clean['Wahlkreis']
elections_1983_df_final['Gebiet'] = elections_1983_df_clean['Gebiet']
elections_1983_df_final['Wahlberechtigte'] = elections_1983_df_clean['Wahlberechtigte'].astype('int64')
elections_1983_df_final['Wähler'] = elections_1983_df_clean['Wähler'].astype('int64')
elections_1983_df_final['Ungültige'] = elections_1983_df_clean['Ungültige'].astype('int64')
elections_1983_df_final['Ungültige:2'] = elections_1983_df_clean['Ungültige.1'].astype('int64')
elections_1983_df_final['Gültige'] = elections_1983_df_clean['Gültige'].astype('int64')
elections_1983_df_final['Gültige:2'] = elections_1983_df_clean['Gültige.1'].astype('int64')
elections_1983_df_final['CDU/CSU'] = elections_1983_df_clean['C D U'].astype('int64') + elections_1983_df_clean['C S U'].astype('int64')
elections_1983_df_final['CDU/CSU:2'] = elections_1983_df_clean['C D U.1'].astype('int64') + elections_1983_df_clean['C S U.1'].astype('int64')
elections_1983_df_final['SPD'] = elections_1983_df_clean['S P D'].astype('int64')
elections_1983_df_final['SPD:2'] = elections_1983_df_clean['S P D.1'].astype('int64')
elections_1983_df_final['FDP'] = elections_1983_df_clean['F.D.P.'].astype('int64')
elections_1983_df_final['FDP:2'] = elections_1983_df_clean['F.D.P..1'].astype('int64')
elections_1983_df_final['GRUENE'] = elections_1983_df_clean['GRUENE'].astype('int64')
elections_1983_df_final['GRUENE:2'] = elections_1983_df_clean['GRUENE.1'].astype('int64')
elections_1983_df_final['KPD'] = elections_1983_df_clean['K P D'].astype('int64') + elections_1983_df_clean['D K P'].astype('int64')
elections_1983_df_final['KPD:2'] = elections_1983_df_clean['K P D.1'].astype('int64') + elections_1983_df_clean['D K P.1'].astype('int64')

#others
elections_1983_df_final['OTHER'] = elections_1983_df_clean['B W K'].astype('int64') + \
                                   elections_1983_df_clean['C.B.V.'].astype('int64') +\
                                   elections_1983_df_clean['E A P'].astype('int64') +\
                                   elections_1983_df_clean['N P D'].astype('int64') +\
                                   elections_1983_df_clean['OE D P'].astype('int64') +\
                                   elections_1983_df_clean['U S D'].astype('int64') +\
                                   elections_1983_df_clean['UEBRIGE'].astype('int64')

#others Zweitstimmen
elections_1983_df_final['OTHER:2'] = elections_1983_df_clean['B W K.1'].astype('int64') + \
                                   elections_1983_df_clean['C.B.V..1'].astype('int64') +\
                                   elections_1983_df_clean['E A P.1'].astype('int64') +\
                                   elections_1983_df_clean['N P D.1'].astype('int64') +\
                                   elections_1983_df_clean['OE D P.1'].astype('int64') +\
                                   elections_1983_df_clean['U S D.1'].astype('int64')

#fill NaNs with 0s
elections_1983_df_final = elections_1983_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1983_df_final['Gültige'] - elections_1983_df_final['CDU/CSU'] - elections_1983_df_final['SPD'] \
    - elections_1983_df_final['FDP'] - elections_1983_df_final['GRUENE'] - elections_1983_df_final['KPD']
B = elections_1983_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1983_df_final['Gültige:2'] - elections_1983_df_final['CDU/CSU:2'] - elections_1983_df_final['SPD:2'] \
    - elections_1983_df_final['FDP:2'] - elections_1983_df_final['GRUENE:2'] - elections_1983_df_final['KPD:2']
B2 = elections_1983_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1983_df_final.to_csv('Bundestagswahlen_clean/elections_1983.csv')

In [33]:
###################### 1987 ###########################

elections_1987_df_clean = elections_1987_df.drop(['Unnamed: 47', 'Unnamed: 48'], axis=1) 

elections_1987_df_clean = elections_1987_df_clean.fillna(0)

#rename unnamed: 1
elections_1987_df_clean = elections_1987_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1987_df_clean = elections_1987_df_clean.drop([0, 1, 14, 23, 56, 61, 134, 158, 176, 215, 262, 269])

#create the final df
elections_1987_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1987_df_final['Wahlkreis'] = elections_1987_df_clean['Wahlkreis']
elections_1987_df_final['Gebiet'] = elections_1987_df_clean['Gebiet']
elections_1987_df_final['Wahlberechtigte'] = elections_1987_df_clean['Wahlberechtigte'].astype('int64')
elections_1987_df_final['Wähler'] = elections_1987_df_clean['Wähler'].astype('int64')
elections_1987_df_final['Ungültige'] = elections_1987_df_clean['Ungültige'].astype('int64')
elections_1987_df_final['Ungültige:2'] = elections_1987_df_clean['Ungültige.1'].astype('int64')
elections_1987_df_final['Gültige'] = elections_1987_df_clean['Gültige'].astype('int64')
elections_1987_df_final['Gültige:2'] = elections_1987_df_clean['Gültige.1'].astype('int64')
elections_1987_df_final['CDU/CSU'] = elections_1987_df_clean['C D U'].astype('int64') + elections_1987_df_clean['C S U'].astype('int64')
elections_1987_df_final['CDU/CSU:2'] = elections_1987_df_clean['C D U.1'].astype('int64') + elections_1987_df_clean['C S U.1'].astype('int64')
elections_1987_df_final['SPD'] = elections_1987_df_clean['S P D'].astype('int64')
elections_1987_df_final['SPD:2'] = elections_1987_df_clean['S P D.1'].astype('int64')
elections_1987_df_final['FDP'] = elections_1987_df_clean['F.D.P.'].astype('int64')
elections_1987_df_final['FDP:2'] = elections_1987_df_clean['F.D.P..1'].astype('int64')
elections_1987_df_final['GRUENE'] = elections_1987_df_clean['GRUENE'].astype('int64')
elections_1987_df_final['GRUENE:2'] = elections_1987_df_clean['GRUENE.1'].astype('int64')

#others
elections_1987_df_final['OTHER'] = elections_1987_df_clean['A S D'].astype('int64') + \
                                   elections_1987_df_clean['B P'].astype('int64') +\
                                   elections_1987_df_clean['C.B.V.'].astype('int64') +\
                                   elections_1987_df_clean['ZENTRUM'].astype('int64') +\
                                   elections_1987_df_clean['MUENDIGE'].astype('int64') +\
                                   elections_1987_df_clean['FRAUEN'].astype('int64') +\
                                   elections_1987_df_clean['F A P'].astype('int64') +\
                                   elections_1987_df_clean['M L P D'].astype('int64') +\
                                   elections_1987_df_clean['N P D'].astype('int64') +\
                                   elections_1987_df_clean['OE D P'].astype('int64') +\
                                   elections_1987_df_clean['PATRIOTEN'].astype('int64') +\
                                   elections_1987_df_clean['FAMILIE'].astype('int64') +\
                                   elections_1987_df_clean['SOLIDARITAET'].astype('int64') +\
                                   elections_1987_df_clean['F S U'].astype('int64') +\
                                   elections_1987_df_clean['H P'].astype('int64') +\
                                   elections_1987_df_clean['U A P'].astype('int64') +\
                                   elections_1987_df_clean['UEBRIGE'].astype('int64')

#others Zweitstimmen
elections_1987_df_final['OTHER:2'] = elections_1987_df_clean['A S D.1'].astype('int64') + \
                                   elections_1987_df_clean['B P.1'].astype('int64') +\
                                   elections_1987_df_clean['C.B.V..1'].astype('int64') +\
                                   elections_1987_df_clean['ZENTRUM.1'].astype('int64') +\
                                   elections_1987_df_clean['MUENDIGE.1'].astype('int64') +\
                                   elections_1987_df_clean['FRAUEN.1'].astype('int64') +\
                                   elections_1987_df_clean['F A P.1'].astype('int64') +\
                                   elections_1987_df_clean['M L P D.1'].astype('int64') +\
                                   elections_1987_df_clean['N P D.1'].astype('int64') +\
                                   elections_1987_df_clean['OE D P.1'].astype('int64') +\
                                   elections_1987_df_clean['PATRIOTEN.1'].astype('int64')

#fill NaNs with 0s
elections_1987_df_final = elections_1987_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1987_df_final['Gültige'] - elections_1987_df_final['CDU/CSU'] - elections_1987_df_final['SPD'] \
 - elections_1987_df_final['FDP'] - elections_1987_df_final['GRUENE'] 
B = elections_1987_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1987_df_final['Gültige:2'] - elections_1987_df_final['CDU/CSU:2'] - elections_1987_df_final['SPD:2'] \
- elections_1987_df_final['FDP:2'] - elections_1987_df_final['GRUENE:2'] 
B2 = elections_1987_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1987_df_final.to_csv('Bundestagswahlen_clean/elections_1987.csv')

In [34]:
###################### 1990 ###########################

elections_1990_df_clean = elections_1990_df #nothing to drop

elections_1990_df_clean = elections_1990_df_clean.fillna(0)

#rename unnamed: 1
elections_1990_df_clean = elections_1990_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1990_df_clean = elections_1990_df_clean.drop([0, 1, 14, 23, 56, 61, 134, 158, 176, 215, 262, 269, 284, 295, 309, 324, 338])

#create the final df
elections_1990_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1990_df_final['Wahlkreis'] = elections_1990_df_clean['Wahlkreis']
elections_1990_df_final['Gebiet'] = elections_1990_df_clean['Gebiet']
elections_1990_df_final['Wahlberechtigte'] = elections_1990_df_clean['Wahlberechtigte'].astype('int64')
elections_1990_df_final['Wähler'] = elections_1990_df_clean['Wähler'].astype('int64')
elections_1990_df_final['Ungültige'] = elections_1990_df_clean['Ungültige'].astype('int64')
elections_1990_df_final['Ungültige:2'] = elections_1990_df_clean['Ungültige.1'].astype('int64')
elections_1990_df_final['Gültige'] = elections_1990_df_clean['Gültige'].astype('int64')
elections_1990_df_final['Gültige:2'] = elections_1990_df_clean['Gültige.1'].astype('int64')
elections_1990_df_final['CDU/CSU'] = elections_1990_df_clean['CDU'].astype('int64') + elections_1990_df_clean['CSU'].astype('int64')
elections_1990_df_final['CDU/CSU:2'] = elections_1990_df_clean['CDU.1'].astype('int64') + elections_1990_df_clean['CSU.1'].astype('int64')
elections_1990_df_final['SPD'] = elections_1990_df_clean['SPD'].astype('int64')
elections_1990_df_final['SPD:2'] = elections_1990_df_clean['SPD.1'].astype('int64')
elections_1990_df_final['FDP'] = elections_1990_df_clean['FDP'].astype('int64')
elections_1990_df_final['FDP:2'] = elections_1990_df_clean['FDP.1'].astype('int64')
elections_1990_df_final['GRUENE'] = elections_1990_df_clean['GRÜNE'].astype('int64') + elections_1990_df_clean['B90/Gr'].astype('int64') 
elections_1990_df_final['GRUENE:2'] = elections_1990_df_clean['GRÜNE.1'].astype('int64') + elections_1990_df_clean['B90/Gr.1'].astype('int64')
elections_1990_df_final['KPD'] = elections_1990_df_clean['KPD'].astype('int64')
elections_1990_df_final['KPD:2'] = elections_1990_df_clean['KPD.1'].astype('int64')
elections_1990_df_final['LINKE'] = elections_1990_df_clean['PDS'].astype('int64')
elections_1990_df_final['LINKE:2'] = elections_1990_df_clean['PDS.1'].astype('int64')

#others
elections_1990_df_final['OTHER'] = elections_1990_df_clean['DSU'].astype('int64') + \
                                   elections_1990_df_clean['BP'].astype('int64') +\
                                   elections_1990_df_clean['DDD'].astype('int64') +\
                                   elections_1990_df_clean['BSA'].astype('int64') +\
                                   elections_1990_df_clean['LIGA'].astype('int64') +\
                                   elections_1990_df_clean['CM'].astype('int64') +\
                                   elections_1990_df_clean['ÖKO-Union'].astype('int64') +\
                                   elections_1990_df_clean['DIE GRAUEN'].astype('int64') +\
                                   elections_1990_df_clean['Mündige'].astype('int64') +\
                                   elections_1990_df_clean['REP'].astype('int64') +\
                                   elections_1990_df_clean['Frauen'].astype('int64') +\
                                   elections_1990_df_clean['NPD'].astype('int64') +\
                                   elections_1990_df_clean['ÖDP'].astype('int64') +\
                                   elections_1990_df_clean['Patrioten'].astype('int64') +\
                                   elections_1990_df_clean['SpAD'].astype('int64') +\
                                   elections_1990_df_clean['VAA'].astype('int64') +\
                                   elections_1990_df_clean['EFP'].astype('int64') +\
                                   elections_1990_df_clean['Übrige'].astype('int64')

#others: Zweitstimmen
elections_1990_df_final['OTHER:2'] = elections_1990_df_clean['DSU.1'].astype('int64') + \
                                   elections_1990_df_clean['BP.1'].astype('int64') +\
                                   elections_1990_df_clean['DDD.1'].astype('int64') +\
                                   elections_1990_df_clean['BSA.1'].astype('int64') +\
                                   elections_1990_df_clean['LIGA.1'].astype('int64') +\
                                   elections_1990_df_clean['CM.1'].astype('int64') +\
                                   elections_1990_df_clean['ÖKO-Union.1'].astype('int64') +\
                                   elections_1990_df_clean['DIE GRAUEN.1'].astype('int64') +\
                                   elections_1990_df_clean['Mündige.1'].astype('int64') +\
                                   elections_1990_df_clean['REP.1'].astype('int64') +\
                                   elections_1990_df_clean['Frauen.1'].astype('int64') +\
                                   elections_1990_df_clean['NPD.1'].astype('int64') +\
                                   elections_1990_df_clean['ÖDP.1'].astype('int64') +\
                                   elections_1990_df_clean['Patrioten.1'].astype('int64') +\
                                   elections_1990_df_clean['SpAD.1'].astype('int64') +\
                                   elections_1990_df_clean['VAA.1'].astype('int64')

#fill NaNs with 0s
elections_1990_df_final = elections_1990_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1990_df_final['Gültige'] - elections_1990_df_final['CDU/CSU'] - elections_1990_df_final['SPD'] \
 - elections_1990_df_final['FDP'] - elections_1990_df_final['GRUENE'] - elections_1990_df_final['KPD'] - elections_1990_df_final['LINKE'] 
B = elections_1990_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1990_df_final['Gültige:2'] - elections_1990_df_final['CDU/CSU:2'] - elections_1990_df_final['SPD:2'] \
 - elections_1990_df_final['FDP:2'] - elections_1990_df_final['GRUENE:2'] - elections_1990_df_final['KPD:2'] - elections_1990_df_final['LINKE:2'] 
B2 = elections_1990_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1990_df_final.to_csv('Bundestagswahlen_clean/elections_1990.csv')

In [35]:
###################### 1994 ###########################

elections_1994_df_clean = elections_1994_df #nothing to drop

elections_1994_df_clean = elections_1994_df_clean.fillna(0)

#rename unnamed: 1
elections_1994_df_clean = elections_1994_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1994_df_clean = elections_1994_df_clean.drop([0, 13, 22, 55, 60, 133, 157, 175, 214, 261, 268, 283, 294, 308, 323, 337, 360])

#create the final df
elections_1994_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1994_df_final['Wahlkreis'] = elections_1994_df_clean['Wahlkreis']
elections_1994_df_final['Gebiet'] = elections_1994_df_clean['Gebiet']
elections_1994_df_final['Wahlberechtigte'] = elections_1994_df_clean['Wahlberechtigte'].astype('int64')
elections_1994_df_final['Wähler'] = elections_1994_df_clean['Wähler'].astype('int64')
elections_1994_df_final['Ungültige'] = elections_1994_df_clean['Ungültige'].astype('int64')
elections_1994_df_final['Ungültige:2'] = elections_1994_df_clean['Ungültige.1'].astype('int64')
elections_1994_df_final['Gültige'] = elections_1994_df_clean['Gültige'].astype('int64')
elections_1994_df_final['Gültige:2'] = elections_1994_df_clean['Gültige.1'].astype('int64')
elections_1994_df_final['CDU/CSU'] = elections_1994_df_clean['CDU'].astype('int64') + elections_1994_df_clean['CSU'].astype('int64')
elections_1994_df_final['CDU/CSU:2'] = elections_1994_df_clean['CDU.1'].astype('int64') + elections_1994_df_clean['CSU.1'].astype('int64')
elections_1994_df_final['SPD'] = elections_1994_df_clean['SPD'].astype('int64')
elections_1994_df_final['SPD:2'] = elections_1994_df_clean['SPD.1'].astype('int64')
elections_1994_df_final['FDP'] = elections_1994_df_clean['FDP'].astype('int64')
elections_1994_df_final['FDP:2'] = elections_1994_df_clean['FDP.1'].astype('int64')
elections_1994_df_final['GRUENE'] = elections_1994_df_clean['GRÜNE'].astype('int64')
elections_1994_df_final['GRUENE:2'] = elections_1994_df_clean['GRÜNE.1'].astype('int64') 
elections_1994_df_final['KPD'] = elections_1994_df_clean['KPD'].astype('int64') + elections_1994_df_clean['DKP'].astype('int64') 
#elections_1994_df_final['KPD:2'] = elections_1994_df_clean['KPD.1'].astype('int64')
elections_1994_df_final['LINKE'] = elections_1994_df_clean['PDS'].astype('int64')
elections_1994_df_final['LINKE:2'] = elections_1994_df_clean['PDS.1'].astype('int64')

#others
elections_1994_df_final['OTHER'] = elections_1994_df_clean['REP'].astype('int64') + \
                                   elections_1994_df_clean['APD'].astype('int64') +\
                                   elections_1994_df_clean['BP'].astype('int64') +\
                                   elections_1994_df_clean['BüSo'].astype('int64') +\
                                   elections_1994_df_clean['BSA'].astype('int64') +\
                                   elections_1994_df_clean['LIGA'].astype('int64') +\
                                   elections_1994_df_clean['CM'].astype('int64') +\
                                   elections_1994_df_clean['ZENTRUM'].astype('int64') +\
                                   elections_1994_df_clean['GRAUE'].astype('int64') +\
                                   elections_1994_df_clean['NATURGESETZ'].astype('int64') +\
                                   elections_1994_df_clean['MLPD'].astype('int64') +\
                                   elections_1994_df_clean['Tierschutz'].astype('int64') +\
                                   elections_1994_df_clean['ödp'].astype('int64') +\
                                   elections_1994_df_clean['PBC'].astype('int64') +\
                                   elections_1994_df_clean['PASS'].astype('int64') +\
                                   elections_1994_df_clean['STATT Partei'].astype('int64') +\
                                   elections_1994_df_clean['BGD'].astype('int64') +\
                                   elections_1994_df_clean['DSU'].astype('int64') +\
                                   elections_1994_df_clean['DVP'].astype('int64') +\
                                   elections_1994_df_clean['DEMOKRATEN'].astype('int64') +\
                                   elections_1994_df_clean['FBU'].astype('int64') +\
                                   elections_1994_df_clean['FSU'].astype('int64') +\
                                   elections_1994_df_clean['LD'].astype('int64') +\
                                   elections_1994_df_clean['UAP'].astype('int64') +\
                                   elections_1994_df_clean['Einzelbewerber'].astype('int64')

#others: Zweitstimmen
elections_1994_df_final['OTHER:2'] = elections_1994_df_clean['REP.1'].astype('int64') + \
                                   elections_1994_df_clean['APD.1'].astype('int64') +\
                                   elections_1994_df_clean['BP.1'].astype('int64') +\
                                   elections_1994_df_clean['BüSo.1'].astype('int64') +\
                                   elections_1994_df_clean['BSA.1'].astype('int64') +\
                                   elections_1994_df_clean['LIGA.1'].astype('int64') +\
                                   elections_1994_df_clean['CM.1'].astype('int64') +\
                                   elections_1994_df_clean['ZENTRUM.1'].astype('int64') +\
                                   elections_1994_df_clean['GRAUE.1'].astype('int64') +\
                                   elections_1994_df_clean['NATURGESETZ.1'].astype('int64') +\
                                   elections_1994_df_clean['MLPD.1'].astype('int64') +\
                                   elections_1994_df_clean['Tierschutz.1'].astype('int64') +\
                                   elections_1994_df_clean['ödp.1'].astype('int64') +\
                                   elections_1994_df_clean['PBC.1'].astype('int64') +\
                                   elections_1994_df_clean['PASS.1'].astype('int64') +\
                                   elections_1994_df_clean['STATT Partei.1'].astype('int64')

#fill NaNs with 0s
elections_1994_df_final = elections_1994_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1994_df_final['Gültige'] - elections_1994_df_final['CDU/CSU'] - elections_1994_df_final['SPD'] \
   - elections_1994_df_final['FDP'] - elections_1994_df_final['GRUENE'] - elections_1994_df_final['KPD'] \
   - elections_1994_df_final['LINKE'] 
B = elections_1994_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1994_df_final['Gültige:2'] - elections_1994_df_final['CDU/CSU:2'] - elections_1994_df_final['SPD:2'] \
   - elections_1994_df_final['FDP:2'] - elections_1994_df_final['GRUENE:2'] - elections_1994_df_final['KPD:2'] \
   - elections_1994_df_final['LINKE:2'] 
B2 = elections_1994_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#add the results of previous years
#TODO

elections_1994_df_final.to_csv('Bundestagswahlen_clean/elections_1994.csv')

In [36]:
###################### 1998 ###########################

elections_1998_df_clean = elections_1998_df #nothing to drop

elections_1998_df_clean = elections_1998_df_clean.fillna(0)

#rename unnamed: 1
elections_1998_df_clean = elections_1998_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_1998_df_clean = elections_1998_df_clean.drop([0])#, 13, 22, 55, 60, 133, 157, 175, 214, 261, 268, 283, 294, 308, 323, 337, 360])

#create the final df
elections_1998_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_1998_df_final['Wahlkreis'] = elections_1998_df_clean['Wahlkreis']
elections_1998_df_final['Gebiet'] = elections_1998_df_clean['Gebiet']
elections_1998_df_final['Wahlberechtigte'] = elections_1998_df_clean['Wahlberechtigte'].astype('int64')
elections_1998_df_final['Wähler'] = elections_1998_df_clean['Wähler'].astype('int64')
elections_1998_df_final['Ungültige'] = elections_1998_df_clean['Ungültige'].astype('int64')
elections_1998_df_final['Ungültige:2'] = elections_1998_df_clean['Ungültige.1'].astype('int64')
elections_1998_df_final['Gültige'] = elections_1998_df_clean['Gültige'].astype('int64')
elections_1998_df_final['Gültige:2'] = elections_1998_df_clean['Gültige.1'].astype('int64')
elections_1998_df_final['CDU/CSU'] = elections_1998_df_clean['CDU'].astype('int64') + elections_1998_df_clean['CSU'].astype('int64')
elections_1998_df_final['CDU/CSU:2'] = elections_1998_df_clean['CDU.1'].astype('int64') + elections_1998_df_clean['CSU.1'].astype('int64')
elections_1998_df_final['SPD'] = elections_1998_df_clean['SPD'].astype('int64')
elections_1998_df_final['SPD:2'] = elections_1998_df_clean['SPD.1'].astype('int64')
elections_1998_df_final['FDP'] = elections_1998_df_clean['F.D.P.'].astype('int64')
elections_1998_df_final['FDP:2'] = elections_1998_df_clean['F.D.P..1'].astype('int64')
elections_1998_df_final['GRUENE'] = elections_1998_df_clean['GRÜNE'].astype('int64')
elections_1998_df_final['GRUENE:2'] = elections_1998_df_clean['GRÜNE.1'].astype('int64') 
elections_1998_df_final['KPD'] = elections_1998_df_clean['DKP'].astype('int64') 
#elections_1998_df_final['KPD:2'] = elections_1998_df_clean['DKP.1'].astype('int64')
elections_1998_df_final['LINKE'] = elections_1998_df_clean['PDS'].astype('int64')
elections_1998_df_final['LINKE:2'] = elections_1998_df_clean['PDS.1'].astype('int64')

#others
elections_1998_df_final['OTHER'] = elections_1998_df_clean['Deutschland'].astype('int64') + \
                                   elections_1998_df_clean['APPD'].astype('int64') +\
                                   elections_1998_df_clean['APD'].astype('int64') +\
                                   elections_1998_df_clean['BP'].astype('int64') +\
                                   elections_1998_df_clean['BüSo'].astype('int64') +\
                                   elections_1998_df_clean['BFB'].astype('int64') +\
                                   elections_1998_df_clean['CHANCE 2000'].astype('int64') +\
                                   elections_1998_df_clean['CM'].astype('int64') +\
                                   elections_1998_df_clean['DPD'].astype('int64') +\
                                   elections_1998_df_clean['GRAUE'].astype('int64') +\
                                   elections_1998_df_clean['REP'].astype('int64') +\
                                   elections_1998_df_clean['FAMILIE'].astype('int64') +\
                                   elections_1998_df_clean['DIE FRAUEN'].astype('int64') +\
                                   elections_1998_df_clean['HP'].astype('int64') +\
                                   elections_1998_df_clean['MLPD'].astype('int64') +\
                                   elections_1998_df_clean['Tierschutz'].astype('int64') +\
                                   elections_1998_df_clean['NPD'].astype('int64') +\
                                   elections_1998_df_clean['NATURGESETZ'].astype('int64') +\
                                   elections_1998_df_clean['FORUM'].astype('int64') +\
                                   elections_1998_df_clean['ödp'].astype('int64') +\
                                   elections_1998_df_clean['PBC'].astype('int64') +\
                                   elections_1998_df_clean['AB 2000'].astype('int64') +\
                                   elections_1998_df_clean['PASS'].astype('int64') +\
                                   elections_1998_df_clean['DSU'].astype('int64') +\
                                   elections_1998_df_clean['ZENTRUM'].astype('int64') +\
                                   elections_1998_df_clean['DMP'].astype('int64') +\
                                   elections_1998_df_clean['FP Deutschl.'].astype('int64') +\
                                   elections_1998_df_clean['FSU'].astype('int64') +\
                                   elections_1998_df_clean['STATT Partei'].astype('int64') +\
                                   elections_1998_df_clean['Übrige'].astype('int64')

#others: Zweitstimmen
elections_1998_df_final['OTHER:2'] = elections_1998_df_clean['Deutschland.1'].astype('int64') + \
                                   elections_1998_df_clean['APPD.1'].astype('int64') +\
                                   elections_1998_df_clean['APD.1'].astype('int64') +\
                                   elections_1998_df_clean['BP.1'].astype('int64') +\
                                   elections_1998_df_clean['BüSo.1'].astype('int64') +\
                                   elections_1998_df_clean['BFB.1'].astype('int64') +\
                                   elections_1998_df_clean['CHANCE 2000.1'].astype('int64') +\
                                   elections_1998_df_clean['CM.1'].astype('int64') +\
                                   elections_1998_df_clean['DPD.1'].astype('int64') +\
                                   elections_1998_df_clean['DVU'].astype('int64') +\
                                   elections_1998_df_clean['GRAUE.1'].astype('int64') +\
                                   elections_1998_df_clean['REP.1'].astype('int64') +\
                                   elections_1998_df_clean['FAMILIE.1'].astype('int64') +\
                                   elections_1998_df_clean['DIE FRAUEN.1'].astype('int64') +\
                                   elections_1998_df_clean['HP.1'].astype('int64') +\
                                   elections_1998_df_clean['Pro DM'].astype('int64') +\
                                   elections_1998_df_clean['MLPD.1'].astype('int64') +\
                                   elections_1998_df_clean['Tierschutz.1'].astype('int64') +\
                                   elections_1998_df_clean['NPD.1'].astype('int64') +\
                                   elections_1998_df_clean['NATURGESETZ.1'].astype('int64') +\
                                   elections_1998_df_clean['FORUM.1'].astype('int64') +\
                                   elections_1998_df_clean['ödp.1'].astype('int64') +\
                                   elections_1998_df_clean['PBC.1'].astype('int64') +\
                                   elections_1998_df_clean['AB 2000.1'].astype('int64') +\
                                   elections_1998_df_clean['Nichtwähler'].astype('int64') +\
                                   elections_1998_df_clean['PSG'].astype('int64') +\
                                   elections_1998_df_clean['PASS.1'].astype('int64') 
                     
#fill NaNs with 0s
elections_1998_df_final = elections_1998_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_1998_df_final['Gültige'] - elections_1998_df_final['CDU/CSU'] - elections_1998_df_final['SPD'] \
   - elections_1998_df_final['FDP'] - elections_1998_df_final['GRUENE'] - elections_1998_df_final['KPD']\
   - elections_1998_df_final['LINKE'] 
B = elections_1998_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_1998_df_final['Gültige:2'] - elections_1998_df_final['CDU/CSU:2'] - elections_1998_df_final['SPD:2'] \
   - elections_1998_df_final['FDP:2'] - elections_1998_df_final['GRUENE:2'] - elections_1998_df_final['LINKE:2'] 
B2 = elections_1998_df_final['OTHER:2']
assert(np.allclose(A2, B2))


#IMPORTANT: something is wrong with the 1998 election data prep. Look into it

#add the results of previous years
#TODO

elections_1998_df_final.to_csv('Bundestagswahlen_clean/elections_1998.csv')

In [37]:
###################### 2002 ###########################

elections_2002_df_clean = elections_2002_df #nothing to drop

elections_2002_df_clean = elections_2002_df_clean.fillna(0)

#rename unnamed: 1
elections_2002_df_clean = elections_2002_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_2002_df_clean = elections_2002_df_clean.drop([0])#, 13, 22, 55, 60, 133, 157, 175, 214, 261, 268, 283, 294, 308, 323, 337, 360])

#create the final df
elections_2002_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_2002_df_final['Wahlkreis'] = elections_2002_df_clean['Wahlkreis']
elections_2002_df_final['Gebiet'] = elections_2002_df_clean['Gebiet']
elections_2002_df_final['Wahlberechtigte'] = elections_2002_df_clean['Wahlberechtigte'].astype('int64')
elections_2002_df_final['Wähler'] = elections_2002_df_clean['Wähler'].astype('int64')
elections_2002_df_final['Ungültige'] = elections_2002_df_clean['Ungültige'].astype('int64')
elections_2002_df_final['Ungültige:2'] = elections_2002_df_clean['Ungültige.1'].astype('int64')
elections_2002_df_final['Gültige'] = elections_2002_df_clean['Gültige'].astype('int64')
elections_2002_df_final['Gültige:2'] = elections_2002_df_clean['Gültige.1'].astype('int64')
elections_2002_df_final['CDU/CSU'] = elections_2002_df_clean['CDU'].astype('int64') + elections_2002_df_clean['CSU'].astype('int64')
elections_2002_df_final['CDU/CSU:2'] = elections_2002_df_clean['CDU.1'].astype('int64') + elections_2002_df_clean['CSU.1'].astype('int64')
elections_2002_df_final['SPD'] = elections_2002_df_clean['SPD'].astype('int64')
elections_2002_df_final['SPD:2'] = elections_2002_df_clean['SPD.1'].astype('int64')
elections_2002_df_final['FDP'] = elections_2002_df_clean['FDP'].astype('int64')
elections_2002_df_final['FDP:2'] = elections_2002_df_clean['FDP.1'].astype('int64')
elections_2002_df_final['GRUENE'] = elections_2002_df_clean['GRÜNE'].astype('int64')
elections_2002_df_final['GRUENE:2'] = elections_2002_df_clean['GRÜNE.1'].astype('int64') 
elections_2002_df_final['KPD'] = elections_2002_df_clean['KPD'].astype('int64') + elections_2002_df_clean['DKP'].astype('int64')
elections_2002_df_final['KPD:2'] = elections_2002_df_clean['KPD.1'].astype('int64')
elections_2002_df_final['LINKE'] = elections_2002_df_clean['PDS'].astype('int64')
elections_2002_df_final['LINKE:2'] = elections_2002_df_clean['PDS.1'].astype('int64')

#others
elections_2002_df_final['OTHER'] = elections_2002_df_clean['REP'].astype('int64') + \
                                   elections_2002_df_clean['GRAUE'].astype('int64') +\
                                   elections_2002_df_clean['Tierschutz'].astype('int64') +\
                                   elections_2002_df_clean['NPD'].astype('int64') +\
                                   elections_2002_df_clean['ödp'].astype('int64') +\
                                   elections_2002_df_clean['PBC'].astype('int64') +\
                                   elections_2002_df_clean['DIE FRAUEN'].astype('int64') +\
                                   elections_2002_df_clean['BP'].astype('int64') +\
                                   elections_2002_df_clean['FAMILIE'].astype('int64') +\
                                   elections_2002_df_clean['CM'].astype('int64') +\
                                   elections_2002_df_clean['BüSo'].astype('int64') +\
                                   elections_2002_df_clean['HP'].astype('int64') +\
                                   elections_2002_df_clean['Violetten'].astype('int64') +\
                                   elections_2002_df_clean['AUFBRUCH'].astype('int64') +\
                                   elections_2002_df_clean['ZENTRUM'].astype('int64') +\
                                   elections_2002_df_clean['PRG'].astype('int64') +\
                                   elections_2002_df_clean['Schill'].astype('int64') +\
                                   elections_2002_df_clean['Deutschland'].astype('int64') +\
                                   elections_2002_df_clean['DSU'].astype('int64') +\
                                   elections_2002_df_clean['FP Deutschl.'].astype('int64') +\
                                   elections_2002_df_clean['Übrige'].astype('int64')

#others: Zweitstimmen
elections_2002_df_final['OTHER:2'] = elections_2002_df_clean['REP.1'].astype('int64') + \
                                   elections_2002_df_clean['GRAUE.1'].astype('int64') +\
                                   elections_2002_df_clean['Tierschutz.1'].astype('int64') +\
                                   elections_2002_df_clean['NPD.1'].astype('int64') +\
                                   elections_2002_df_clean['ödp.1'].astype('int64') +\
                                   elections_2002_df_clean['PBC.1'].astype('int64') +\
                                   elections_2002_df_clean['DIE FRAUEN.1'].astype('int64') +\
                                   elections_2002_df_clean['BP.1'].astype('int64') +\
                                   elections_2002_df_clean['FAMILIE.1'].astype('int64') +\
                                   elections_2002_df_clean['CM.1'].astype('int64') +\
                                   elections_2002_df_clean['BüSo.1'].astype('int64') +\
                                   elections_2002_df_clean['HP.1'].astype('int64') +\
                                   elections_2002_df_clean['Violetten.1'].astype('int64') +\
                                   elections_2002_df_clean['AUFBRUCH.1'].astype('int64') +\
                                   elections_2002_df_clean['ZENTRUM.1'].astype('int64') +\
                                   elections_2002_df_clean['PRG.1'].astype('int64') +\
                                   elections_2002_df_clean['Schill.1'].astype('int64')

#fill NaNs with 0s
elections_2002_df_final = elections_2002_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_2002_df_final['Gültige'] - elections_2002_df_final['CDU/CSU'] - elections_2002_df_final['SPD'] \
 - elections_2002_df_final['FDP'] - elections_2002_df_final['GRUENE'] - elections_2002_df_final['KPD'] - elections_2002_df_final['LINKE'] 
B = elections_2002_df_final['OTHER']
#print(A-B)
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_2002_df_final['Gültige:2'] - elections_2002_df_final['CDU/CSU:2'] - elections_2002_df_final['SPD:2'] \
   - elections_2002_df_final['FDP:2'] - elections_2002_df_final['GRUENE:2'] - elections_2002_df_final['KPD:2'] \
   - elections_2002_df_final['LINKE:2'] 
B2 = elections_2002_df_final['OTHER:2']
#print(A2-B2)
assert(np.allclose(A2, B2))

#add the results of previous years
#TODO

elections_2002_df_final.to_csv('Bundestagswahlen_clean/elections_2002.csv')

In [38]:
###################### 2005 ###########################

elections_2005_df_clean = elections_2005_df#.drop(['Unnamed: 147'], axis=1)

elections_2005_df_clean = elections_2005_df_clean.fillna(0)

#rename unnamed: 1
elections_2005_df_clean = elections_2005_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_2005_df_clean = elections_2005_df_clean.drop([0,1])#, 13, 22, 55, 60, 133, 157, 175, 214, 261, 268, 283, 294, 308, 323, 337, 360])

#create the final df
elections_2005_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_2005_df_final['Wahlkreis'] = elections_2005_df_clean['Nr']
elections_2005_df_final['Gebiet'] = elections_2005_df_clean['Gebiet']
elections_2005_df_final['Wahlberechtigte'] = elections_2005_df_clean['Wahlberechtigte'].astype('int64')
elections_2005_df_final['Wähler'] = elections_2005_df_clean['Wähler'].astype('int64')
elections_2005_df_final['Ungültige'] = elections_2005_df_clean['Ungültige'].astype('int64')
elections_2005_df_final['Ungültige:2'] = elections_2005_df_clean['Unnamed: 13'].astype('int64')
elections_2005_df_final['Gültige'] = elections_2005_df_clean['Gültige'].astype('int64')
elections_2005_df_final['Gültige:2'] = elections_2005_df_clean['Unnamed: 17'].astype('int64')
elections_2005_df_final['CDU/CSU'] = elections_2005_df_clean['CDU'].astype('int64') + elections_2005_df_clean['CSU'].astype('int64')
elections_2005_df_final['CDU/CSU_prev'] = elections_2005_df_clean['Unnamed: 24'].astype('int64') + elections_2005_df_clean['Unnamed: 28'].astype('int64')
elections_2005_df_final['CDU/CSU:2'] = elections_2005_df_clean['Unnamed: 25'].astype('int64') + elections_2005_df_clean['Unnamed: 29'].astype('int64')
elections_2005_df_final['CDU/CSU_prev'] = elections_2005_df_clean['Unnamed: 26'].astype('int64') + elections_2005_df_clean['Unnamed: 30'].astype('int64')
elections_2005_df_final['SPD'] = elections_2005_df_clean['SPD'].astype('int64')
elections_2005_df_final['SPD_prev'] = elections_2005_df_clean['Unnamed: 20'].astype('int64')
elections_2005_df_final['SPD:2'] = elections_2005_df_clean['Unnamed: 21'].astype('int64')
elections_2005_df_final['SPD:2_prev'] = elections_2005_df_clean['Unnamed: 22'].astype('int64')
elections_2005_df_final['FDP'] = elections_2005_df_clean['FDP'].astype('int64')
elections_2005_df_final['FDP_prev'] = elections_2005_df_clean['Unnamed: 36'].astype('int64')
elections_2005_df_final['FDP:2'] = elections_2005_df_clean['Unnamed: 37'].astype('int64')
elections_2005_df_final['FDP:2_prev'] = elections_2005_df_clean['Unnamed: 38'].astype('int64')
elections_2005_df_final['GRUENE'] = elections_2005_df_clean['GRÜNE'].astype('int64')
elections_2005_df_final['GRUENE_prev'] = elections_2005_df_clean['Unnamed: 32'].astype('int64')
elections_2005_df_final['GRUENE:2'] = elections_2005_df_clean['Unnamed: 33'].astype('int64')
elections_2005_df_final['GRUENE:2_prev'] = elections_2005_df_clean['Unnamed: 34'].astype('int64')
#elections_2005_df_final['KPD'] = elections_2005_df_clean['KPD'].astype('int64')
#elections_2005_df_final['KPD:2'] = elections_2005_df_clean['KPD.1'].astype('int64')
elections_2005_df_final['LINKE'] = elections_2005_df_clean['Die Linke.'].astype('int64')
elections_2005_df_final['LINKE_prev'] = elections_2005_df_clean['Unnamed: 40'].astype('int64')
elections_2005_df_final['LINKE:2'] = elections_2005_df_clean['Unnamed: 41'].astype('int64')
elections_2005_df_final['LINKE:2_prev'] = elections_2005_df_clean['Unnamed: 42'].astype('int64')

#others
elections_2005_df_final['OTHER'] = elections_2005_df_clean['Offensive D'].astype('int64') + \
                                   elections_2005_df_clean['REP'].astype('int64') +\
                                   elections_2005_df_clean['NPD'].astype('int64') +\
                                   elections_2005_df_clean['Die Tierschutzpartei'].astype('int64') +\
                                   elections_2005_df_clean['GRAUE'].astype('int64') +\
                                   elections_2005_df_clean['PBC'].astype('int64') +\
                                   elections_2005_df_clean['DIE FRAUEN'].astype('int64') +\
                                   elections_2005_df_clean['FAMILIE'].astype('int64') +\
                                   elections_2005_df_clean['BüSo'].astype('int64') +\
                                   elections_2005_df_clean['BP'].astype('int64') +\
                                   elections_2005_df_clean['ZENTRUM'].astype('int64') +\
                                   elections_2005_df_clean['Deutschland'].astype('int64') +\
                                   elections_2005_df_clean['AGFG'].astype('int64') +\
                                   elections_2005_df_clean['APPD'].astype('int64') +\
                                   elections_2005_df_clean['50Plus'].astype('int64') +\
                                   elections_2005_df_clean['MLPD'].astype('int64') +\
                                   elections_2005_df_clean['Die PARTEI'].astype('int64') +\
                                   elections_2005_df_clean['PSG'].astype('int64') +\
                                   elections_2005_df_clean['Pro DM'].astype('int64') +\
                                   elections_2005_df_clean['CM'].astype('int64') +\
                                   elections_2005_df_clean['DSU'].astype('int64') +\
                                   elections_2005_df_clean['HP'].astype('int64') +\
                                   elections_2005_df_clean['HUMANWIRTSCHAFTS- PARTEI'].astype('int64') +\
                                   elections_2005_df_clean['STATT Partei'].astype('int64') +\
                                   elections_2005_df_clean['UNABHÄNGIGE'].astype('int64') +\
                                   elections_2005_df_clean['Übrige'].astype('int64')

#others previous election results
elections_2005_df_final['OTHER_prev'] = elections_2005_df_clean['Unnamed: 44'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 48'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 52'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 56'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 60'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 64'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 68'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 72'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 76'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 80'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 84'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 88'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 92'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 96'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 100'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 104'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 108'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 112'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 116'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 120'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 124'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 128'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 132'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 136'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 140'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 144'].astype('int64')

#others Zweitstimmen
elections_2005_df_final['OTHER:2'] =    elections_2005_df_clean['Unnamed: 45'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 49'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 53'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 57'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 61'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 65'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 69'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 73'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 77'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 81'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 85'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 89'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 93'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 97'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 101'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 105'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 109'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 113'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 117'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 121'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 125'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 129'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 133'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 137'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 141'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 145'].astype('int64') 

#others Zweitstimmen previous election
elections_2005_df_final['OTHER:2_prev'] = elections_2005_df_clean['Unnamed: 46'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 50'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 54'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 58'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 62'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 66'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 70'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 74'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 78'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 82'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 86'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 90'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 94'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 98'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 102'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 106'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 110'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 114'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 118'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 122'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 126'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 130'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 134'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 138'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 142'].astype('int64') + \
                                        elections_2005_df_clean['Unnamed: 146'].astype('int64') 
                                        
    
#fill NaNs with 0s
elections_2005_df_final = elections_2005_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_2005_df_final['Gültige'] - elections_2005_df_final['CDU/CSU'] - elections_2005_df_final['SPD'] \
 - elections_2005_df_final['FDP'] - elections_2005_df_final['GRUENE'] - elections_2005_df_final['KPD'] - elections_2005_df_final['LINKE'] 
B = elections_2005_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_2005_df_final['Gültige:2'] - elections_2005_df_final['CDU/CSU:2'] - elections_2005_df_final['SPD:2'] \
 - elections_2005_df_final['FDP:2'] - elections_2005_df_final['GRUENE:2'] - elections_2005_df_final['KPD:2'] - elections_2005_df_final['LINKE:2'] 
B2 = elections_2005_df_final['OTHER:2']
assert(np.allclose(A2, B2))


elections_2005_df_final.to_csv('Bundestagswahlen_clean/elections_2005.csv')

In [39]:
###################### 2009 ###########################

elections_2009_df_clean = elections_2009_df#.drop(['Unnamed: 135'], axis=1)

elections_2009_df_clean = elections_2009_df_clean.fillna(0)

#rename unnamed: 1
elections_2009_df_clean = elections_2009_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_2009_df_clean = elections_2009_df_clean.drop([0,1])

#create the final df
elections_2009_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_2009_df_final['Wahlkreis'] = elections_2009_df_clean['Nr']
elections_2009_df_final['Gebiet'] = elections_2009_df_clean['Gebiet']
elections_2009_df_final['Wahlberechtigte'] = elections_2009_df_clean['Wahlberechtigte'].astype('int64')
elections_2009_df_final['Wähler'] = elections_2009_df_clean['Wähler'].astype('int64')
elections_2009_df_final['Ungültige'] = elections_2009_df_clean['Ungültige'].astype('int64')
elections_2009_df_final['Ungültige:2'] = elections_2009_df_clean['Unnamed: 13'].astype('int64')
elections_2009_df_final['Gültige'] = elections_2009_df_clean['Gültige'].astype('int64')
elections_2009_df_final['Gültige:2'] = elections_2009_df_clean['Unnamed: 17'].astype('int64')
elections_2009_df_final['CDU/CSU'] = elections_2009_df_clean['CDU'].astype('int64') + elections_2009_df_clean['CSU'].astype('int64')
elections_2009_df_final['CDU/CSU_prev'] = elections_2009_df_clean['Unnamed: 24'].astype('int64') + elections_2009_df_clean['Unnamed: 40'].astype('int64')
elections_2009_df_final['CDU/CSU:2'] = elections_2009_df_clean['Unnamed: 25'].astype('int64') + elections_2009_df_clean['Unnamed: 41'].astype('int64')
elections_2009_df_final['CDU/CSU_prev'] = elections_2009_df_clean['Unnamed: 26'].astype('int64') + elections_2009_df_clean['Unnamed: 42'].astype('int64')
elections_2009_df_final['SPD'] = elections_2009_df_clean['SPD'].astype('int64')
elections_2009_df_final['SPD_prev'] = elections_2009_df_clean['Unnamed: 20'].astype('int64')
elections_2009_df_final['SPD:2'] = elections_2009_df_clean['Unnamed: 21'].astype('int64')
elections_2009_df_final['SPD:2_prev'] = elections_2009_df_clean['Unnamed: 22'].astype('int64')
elections_2009_df_final['FDP'] = elections_2009_df_clean['FDP'].astype('int64')
elections_2009_df_final['FDP_prev'] = elections_2009_df_clean['Unnamed: 28'].astype('int64')
elections_2009_df_final['FDP:2'] = elections_2009_df_clean['Unnamed: 29'].astype('int64')
elections_2009_df_final['FDP:2_prev'] = elections_2009_df_clean['Unnamed: 30'].astype('int64')
elections_2009_df_final['GRUENE'] = elections_2009_df_clean['GRÜNE'].astype('int64')
elections_2009_df_final['GRUENE_prev'] = elections_2009_df_clean['Unnamed: 36'].astype('int64')
elections_2009_df_final['GRUENE:2'] = elections_2009_df_clean['Unnamed: 37'].astype('int64')
elections_2009_df_final['GRUENE:2_prev'] = elections_2009_df_clean['Unnamed: 38'].astype('int64')
elections_2009_df_final['KPD'] = elections_2009_df_clean['DKP'].astype('int64')
elections_2009_df_final['KPD_prev'] = elections_2009_df_clean['Unnamed: 96'].astype('int64')
elections_2009_df_final['KPD:2'] = elections_2009_df_clean['Unnamed: 97'].astype('int64')
elections_2009_df_final['KPD:2_prev'] = elections_2009_df_clean['Unnamed: 98'].astype('int64')
elections_2009_df_final['LINKE'] = elections_2009_df_clean['DIE LINKE'].astype('int64')
elections_2009_df_final['LINKE_prev'] = elections_2009_df_clean['Unnamed: 32'].astype('int64')
elections_2009_df_final['LINKE:2'] = elections_2009_df_clean['Unnamed: 33'].astype('int64')
elections_2009_df_final['LINKE:2_prev'] = elections_2009_df_clean['Unnamed: 34'].astype('int64')

#others
elections_2009_df_final['OTHER'] = elections_2009_df_clean['NPD'].astype('int64') + \
                                   elections_2009_df_clean['REP'].astype('int64') +\
                                   elections_2009_df_clean['FAMILIE'].astype('int64') +\
                                   elections_2009_df_clean['Die Tierschutzpartei'].astype('int64') +\
                                   elections_2009_df_clean['PBC'].astype('int64') +\
                                   elections_2009_df_clean['MLPD'].astype('int64') +\
                                   elections_2009_df_clean['BüSo'].astype('int64') +\
                                   elections_2009_df_clean['BP'].astype('int64') +\
                                   elections_2009_df_clean['PSG'].astype('int64') +\
                                   elections_2009_df_clean['Volksabstimmung'].astype('int64') +\
                                   elections_2009_df_clean['ZENTRUM'].astype('int64') +\
                                   elections_2009_df_clean['ADM'].astype('int64') +\
                                   elections_2009_df_clean['CM'].astype('int64') +\
                                   elections_2009_df_clean['DVU'].astype('int64') +\
                                   elections_2009_df_clean['DIE VIOLETTEN'].astype('int64') +\
                                   elections_2009_df_clean['FWD'].astype('int64') +\
                                   elections_2009_df_clean['ödp'].astype('int64') +\
                                   elections_2009_df_clean['PIRATEN'].astype('int64') +\
                                   elections_2009_df_clean['RRP'].astype('int64') +\
                                   elections_2009_df_clean['RENTNER'].astype('int64') +\
                                   elections_2009_df_clean['Freie Union'].astype('int64') +\
                                   elections_2009_df_clean['Übrige'].astype('int64')

#others previous election results
elections_2009_df_final['OTHER_prev'] = elections_2009_df_clean['Unnamed: 44'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 48'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 52'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 56'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 60'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 64'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 68'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 72'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 76'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 80'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 84'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 88'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 92'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 100'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 104'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 108'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 112'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 116'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 120'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 124'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 128'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 132'].astype('int64')

#others Zweitstimmen
elections_2009_df_final['OTHER:2'] =    elections_2009_df_clean['Unnamed: 45'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 49'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 53'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 57'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 61'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 65'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 69'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 73'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 77'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 81'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 85'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 89'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 93'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 101'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 105'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 109'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 113'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 117'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 121'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 125'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 129'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 133'].astype('int64')

#others Zweitstimmen previous election
elections_2009_df_final['OTHER:2_prev'] = elections_2009_df_clean['Unnamed: 46'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 50'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 54'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 58'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 62'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 66'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 70'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 74'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 78'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 82'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 86'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 90'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 94'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 102'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 106'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 110'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 114'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 118'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 122'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 126'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 130'].astype('int64') + \
                                        elections_2009_df_clean['Unnamed: 134'].astype('int64') 

#fill NaNs with 0s
elections_2009_df_final = elections_2009_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_2009_df_final['Gültige'] - elections_2009_df_final['CDU/CSU'] - elections_2009_df_final['SPD'] \
  - elections_2009_df_final['FDP'] - elections_2009_df_final['GRUENE'] - elections_2009_df_final['KPD'] \
  - elections_2009_df_final['LINKE'] 
B = elections_2009_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_2009_df_final['Gültige:2'] - elections_2009_df_final['CDU/CSU:2'] - elections_2009_df_final['SPD:2'] \
   - elections_2009_df_final['FDP:2'] - elections_2009_df_final['GRUENE:2'] - elections_2009_df_final['KPD:2'] \
   - elections_2009_df_final['LINKE:2'] 
B2 = elections_2009_df_final['OTHER:2']
assert(np.allclose(A2, B2))


elections_2009_df_final.to_csv('Bundestagswahlen_clean/elections_2009.csv')

In [40]:
###################### 2013 ###########################

elections_2013_df_clean = elections_2013_df#.drop(['Unnamed: 135'], axis=1)

elections_2013_df_clean = elections_2013_df_clean.fillna(0)

#rename unnamed: 1
elections_2013_df_clean = elections_2013_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_2013_df_clean = elections_2013_df_clean.drop([0,1])

#create the final df
elections_2013_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_2013_df_final['Wahlkreis'] = elections_2013_df_clean['Nr']
elections_2013_df_final['Gebiet'] = elections_2013_df_clean['Gebiet']
elections_2013_df_final['Wahlberechtigte'] = elections_2013_df_clean['Wahlberechtigte'].astype('int64')
elections_2013_df_final['Wähler'] = elections_2013_df_clean['Wähler'].astype('int64')
elections_2013_df_final['Ungültige'] = elections_2013_df_clean['Ungültige'].astype('int64')
elections_2013_df_final['Ungültige:2'] = elections_2013_df_clean['Unnamed: 13'].astype('int64')
elections_2013_df_final['Gültige'] = elections_2013_df_clean['Gültige'].astype('int64')
elections_2013_df_final['Gültige:2'] = elections_2013_df_clean['Unnamed: 17'].astype('int64')
elections_2013_df_final['CDU/CSU'] = elections_2013_df_clean['CDU'].astype('int64') + elections_2013_df_clean['CSU'].astype('int64')
elections_2013_df_final['CDU/CSU_prev'] = elections_2013_df_clean['Unnamed: 20'].astype('int64') + elections_2013_df_clean['Unnamed: 40'].astype('int64')
elections_2013_df_final['CDU/CSU:2'] = elections_2013_df_clean['Unnamed: 21'].astype('int64') + elections_2013_df_clean['Unnamed: 41'].astype('int64')
elections_2013_df_final['CDU/CSU_prev'] = elections_2013_df_clean['Unnamed: 22'].astype('int64') + elections_2013_df_clean['Unnamed: 42'].astype('int64')
elections_2013_df_final['SPD'] = elections_2013_df_clean['SPD'].astype('int64')
elections_2013_df_final['SPD_prev'] = elections_2013_df_clean['Unnamed: 24'].astype('int64')
elections_2013_df_final['SPD:2'] = elections_2013_df_clean['Unnamed: 25'].astype('int64')
elections_2013_df_final['SPD:2_prev'] = elections_2013_df_clean['Unnamed: 26'].astype('int64')
elections_2013_df_final['FDP'] = elections_2013_df_clean['FDP'].astype('int64')
elections_2013_df_final['FDP_prev'] = elections_2013_df_clean['Unnamed: 28'].astype('int64')
elections_2013_df_final['FDP:2'] = elections_2013_df_clean['Unnamed: 29'].astype('int64')
elections_2013_df_final['FDP:2_prev'] = elections_2013_df_clean['Unnamed: 30'].astype('int64')
elections_2013_df_final['GRUENE'] = elections_2013_df_clean['GRÜNE'].astype('int64')
elections_2013_df_final['GRUENE_prev'] = elections_2013_df_clean['Unnamed: 36'].astype('int64')
elections_2013_df_final['GRUENE:2'] = elections_2013_df_clean['Unnamed: 37'].astype('int64')
elections_2013_df_final['GRUENE:2_prev'] = elections_2013_df_clean['Unnamed: 38'].astype('int64')
elections_2013_df_final['LINKE'] = elections_2013_df_clean['DIE LINKE'].astype('int64')
elections_2013_df_final['LINKE_prev'] = elections_2013_df_clean['Unnamed: 32'].astype('int64')
elections_2013_df_final['LINKE:2'] = elections_2013_df_clean['Unnamed: 33'].astype('int64')
elections_2013_df_final['LINKE:2_prev'] = elections_2013_df_clean['Unnamed: 34'].astype('int64')
elections_2013_df_final['AFD'] = elections_2013_df_clean['AfD'].astype('int64')
elections_2013_df_final['AFD_prev'] = elections_2013_df_clean['Unnamed: 104'].astype('int64')
elections_2013_df_final['AFD:2'] = elections_2013_df_clean['Unnamed: 105'].astype('int64')
elections_2013_df_final['AFD:2_prev'] = elections_2013_df_clean['Unnamed: 106'].astype('int64')
elections_2013_df_final['KPD'] = elections_2013_df_clean['DKP'].astype('int64')
elections_2013_df_final['KPD_prev'] = elections_2013_df_clean['Unnamed: 148'].astype('int64')
elections_2013_df_final['KPD:2'] = elections_2013_df_clean['Unnamed: 149'].astype('int64')
elections_2013_df_final['KPD:2_prev'] = elections_2013_df_clean['Unnamed: 150'].astype('int64')


#others
elections_2013_df_final['OTHER'] = elections_2013_df_clean['PIRATEN'].astype('int64') + \
                                   elections_2013_df_clean['NPD'].astype('int64') +\
                                   elections_2013_df_clean['Tierschutzpartei'].astype('int64') +\
                                   elections_2013_df_clean['REP'].astype('int64') +\
                                   elections_2013_df_clean['ÖDP'].astype('int64') +\
                                   elections_2013_df_clean['FAMILIE'].astype('int64') +\
                                   elections_2013_df_clean['Bündnis 21/RRP'].astype('int64') +\
                                   elections_2013_df_clean['RENTNER'].astype('int64') +\
                                   elections_2013_df_clean['BP'].astype('int64') +\
                                   elections_2013_df_clean['PBC'].astype('int64') +\
                                   elections_2013_df_clean['BüSo'].astype('int64') +\
                                   elections_2013_df_clean['DIE VIOLETTEN'].astype('int64') +\
                                   elections_2013_df_clean['MLPD'].astype('int64') +\
                                   elections_2013_df_clean['Volksabstimmung'].astype('int64') +\
                                   elections_2013_df_clean['PSG'].astype('int64') +\
                                   elections_2013_df_clean['BIG'].astype('int64') +\
                                   elections_2013_df_clean['pro Deutschland'].astype('int64') +\
                                   elections_2013_df_clean['DIE RECHTE'].astype('int64') +\
                                   elections_2013_df_clean['DIE FRAUEN'].astype('int64') +\
                                   elections_2013_df_clean['FREIE WÄHLER'].astype('int64') +\
                                   elections_2013_df_clean['Nichtwähler'].astype('int64') +\
                                   elections_2013_df_clean['PARTEI DER VERNUNFT'].astype('int64') +\
                                   elections_2013_df_clean['Die PARTEI'].astype('int64') +\
                                   elections_2013_df_clean['B'].astype('int64') +\
                                   elections_2013_df_clean['BGD'].astype('int64') +\
                                   elections_2013_df_clean['NEIN!'].astype('int64') +\
                                   elections_2013_df_clean['Übrige'].astype('int64')

#others previous election results
elections_2013_df_final['OTHER_prev'] = elections_2013_df_clean['Unnamed: 44'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 48'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 52'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 56'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 60'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 64'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 68'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 72'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 76'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 80'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 84'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 88'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 92'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 96'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 100'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 108'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 112'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 116'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 120'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 124'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 128'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 132'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 136'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 140'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 144'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 152'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 156'].astype('int64') 

#others Zweitstimmen
elections_2013_df_final['OTHER:2'] =    elections_2013_df_clean['Unnamed: 45'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 49'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 53'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 57'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 61'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 65'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 69'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 73'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 77'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 81'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 85'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 89'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 93'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 97'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 101'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 109'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 113'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 117'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 121'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 125'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 129'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 133'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 137'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 141'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 145'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 153'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 157'].astype('int64') 

#others Zweitstimmen previous election
elections_2013_df_final['OTHER:2_prev'] = elections_2013_df_clean['Unnamed: 46'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 50'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 54'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 58'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 62'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 66'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 70'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 74'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 78'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 82'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 86'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 90'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 94'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 98'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 102'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 110'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 114'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 118'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 122'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 126'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 130'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 134'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 138'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 142'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 146'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 154'].astype('int64') + \
                                        elections_2013_df_clean['Unnamed: 158'].astype('int64') 

#fill NaNs with 0s
elections_2013_df_final = elections_2013_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_2013_df_final['Gültige'] - elections_2013_df_final['CDU/CSU'] - elections_2013_df_final['SPD'] \
  - elections_2013_df_final['AFD'] - elections_2013_df_final['FDP'] - elections_2013_df_final['GRUENE'] \
  - elections_2013_df_final['KPD'] - elections_2013_df_final['LINKE'] 
B = elections_2013_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_2013_df_final['Gültige:2'] - elections_2013_df_final['CDU/CSU:2'] - elections_2013_df_final['SPD:2'] \
   - elections_2013_df_final['AFD:2'] - elections_2013_df_final['FDP:2'] - elections_2013_df_final['GRUENE:2'] \
   - elections_2013_df_final['KPD:2'] - elections_2013_df_final['LINKE:2'] 
B2 = elections_2013_df_final['OTHER:2']
assert(np.allclose(A2, B2))

# IMPORTANT: there is an assertion error somehow but I don't know why; recheck


elections_2013_df_final.to_csv('Bundestagswahlen_clean/elections_2013.csv')

In [41]:
###################### 2017 ###########################

elections_2017_df_clean = elections_2017_df

elections_2017_df_clean = elections_2017_df_clean.fillna(0)

#rename unnamed: 1
#elections_2017_df_clean = elections_2017_df_clean.rename(columns={'Unnamed: 1':'Gebiet'})

#remove the spacing rows
elections_2017_df_clean = elections_2017_df_clean.drop([0,1])

#create the final df
elections_2017_df_final = pd.DataFrame(columns=['Wahlkreis', 'Gebiet', 'Wahlberechtigte', 'Wähler',
                        'Ungültige', 'Ungültige:2', 'Gültige', 'Gültige:2', 'CDU/CSU', 'CDU/CSU_prev', 'CDU/CSU:2', 'CDU/CSU:2_prev',
                        'SPD', 'SPD_prev', 'SPD:2', 'SPD:2_prev', 'FDP', 'FDP_prev', 'FDP:2', 
                        'FDP:2_prev', 'GRUENE', 'GRUENE_prev', 'GRUENE:2', 'GRUENE:2_prev', 'LINKE',
                        'LINKE_prev', 'LINKE:2', 'LINKE:2_prev', 'AFD', 'AFD_prev', 'AFD:2', 'AFD:2_prev',
                        'KPD', 'KPD_prev', 'KPD:2', 'KPD:2_prev', 'BHE', 'BHE_prev', 'BHE:2', 'BHE:2_prev', 
                                'OTHER', 'OTHER_prev', 'OTHER:2', 'OTHER:2_prev'])

#fill the final table with the respective values
elections_2017_df_final['Wahlkreis'] = elections_2017_df_clean['Nr']
elections_2017_df_final['Gebiet'] = elections_2017_df_clean['Gebiet']
elections_2017_df_final['Wahlberechtigte'] = elections_2017_df_clean['Wahlberechtigte'].astype('int64')
elections_2017_df_final['Wähler'] = elections_2017_df_clean['Wähler'].astype('int64')
elections_2017_df_final['Ungültige'] = elections_2017_df_clean['Ungültige'].astype('int64')
elections_2017_df_final['Ungültige:2'] = elections_2017_df_clean['Unnamed: 13'].astype('int64')
elections_2017_df_final['Gültige'] = elections_2017_df_clean['Gültige'].astype('int64')
elections_2017_df_final['Gültige:2'] = elections_2017_df_clean['Unnamed: 17'].astype('int64')
elections_2017_df_final['CDU/CSU'] = elections_2017_df_clean['Christlich Demokratische Union Deutschlands'].astype('int64') \
                                   + elections_2017_df_clean['Christlich-Soziale Union in Bayern e.V.'].astype('int64')
elections_2017_df_final['CDU/CSU_prev'] = elections_2017_df_clean['Unnamed: 20'].astype('int64') \
                                        + elections_2017_df_clean['Unnamed: 36'].astype('int64')
elections_2017_df_final['CDU/CSU:2'] = elections_2017_df_clean['Unnamed: 21'].astype('int64') \
                                     + elections_2017_df_clean['Unnamed: 37'].astype('int64')
elections_2017_df_final['CDU/CSU_prev'] = elections_2017_df_clean['Unnamed: 22'].astype('int64') \
                                        + elections_2017_df_clean['Unnamed: 38'].astype('int64')
elections_2017_df_final['SPD'] = elections_2017_df_clean['Sozialdemokratische Partei Deutschlands'].astype('int64')
elections_2017_df_final['SPD_prev'] = elections_2017_df_clean['Unnamed: 24'].astype('int64')
elections_2017_df_final['SPD:2'] = elections_2017_df_clean['Unnamed: 25'].astype('int64')
elections_2017_df_final['SPD:2_prev'] = elections_2017_df_clean['Unnamed: 26'].astype('int64')
elections_2017_df_final['FDP'] = elections_2017_df_clean['Freie Demokratische Partei'].astype('int64')
elections_2017_df_final['FDP_prev'] = elections_2017_df_clean['Unnamed: 40'].astype('int64')
elections_2017_df_final['FDP:2'] = elections_2017_df_clean['Unnamed: 41'].astype('int64')
elections_2017_df_final['FDP:2_prev'] = elections_2017_df_clean['Unnamed: 42'].astype('int64')
elections_2017_df_final['GRUENE'] = elections_2017_df_clean['BÜNDNIS 90/DIE GRÜNEN'].astype('int64')
elections_2017_df_final['GRUENE_prev'] = elections_2017_df_clean['Unnamed: 32'].astype('int64')
elections_2017_df_final['GRUENE:2'] = elections_2017_df_clean['Unnamed: 33'].astype('int64')
elections_2017_df_final['GRUENE:2_prev'] = elections_2017_df_clean['Unnamed: 34'].astype('int64')
elections_2017_df_final['LINKE'] = elections_2017_df_clean['DIE LINKE'].astype('int64')
elections_2017_df_final['LINKE_prev'] = elections_2017_df_clean['Unnamed: 28'].astype('int64')
elections_2017_df_final['LINKE:2'] = elections_2017_df_clean['Unnamed: 29'].astype('int64')
elections_2017_df_final['LINKE:2_prev'] = elections_2017_df_clean['Unnamed: 30'].astype('int64')
elections_2017_df_final['AFD'] = elections_2017_df_clean['Alternative für Deutschland'].astype('int64')
elections_2017_df_final['AFD_prev'] = elections_2017_df_clean['Unnamed: 44'].astype('int64')
elections_2017_df_final['AFD:2'] = elections_2017_df_clean['Unnamed: 45'].astype('int64')
elections_2017_df_final['AFD:2_prev'] = elections_2017_df_clean['Unnamed: 46'].astype('int64')
elections_2017_df_final['KPD'] = elections_2017_df_clean['Deutsche Kommunistische Partei'].astype('int64')
elections_2017_df_final['KPD_prev'] = elections_2017_df_clean['Unnamed: 120'].astype('int64')
elections_2017_df_final['KPD:2'] = elections_2017_df_clean['Unnamed: 121'].astype('int64')
elections_2017_df_final['KPD:2_prev'] = elections_2017_df_clean['Unnamed: 122'].astype('int64')

#others
elections_2017_df_final['OTHER'] = elections_2017_df_clean['Piratenpartei Deutschland'].astype('int64') + \
                                   elections_2017_df_clean['Nationaldemokratische Partei Deutschlands'].astype('int64') +\
                                   elections_2017_df_clean['FREIE WÄHLER'].astype('int64') +\
                                   elections_2017_df_clean['PARTEI MENSCH UMWELT TIERSCHUTZ'].astype('int64') +\
                                   elections_2017_df_clean['Ökologisch-Demokratische Partei'].astype('int64') +\
                                   elections_2017_df_clean['Partei für Arbeit, Rechtsstaat, Tierschutz, Elitenförderung und basisdemokratische Initiative'].astype('int64') +\
                                   elections_2017_df_clean['Bayernpartei'].astype('int64') +\
                                   elections_2017_df_clean['Ab jetzt...Demokratie durch Volksabstimmung'].astype('int64') +\
                                   elections_2017_df_clean['Partei der Vernunft'].astype('int64') +\
                                   elections_2017_df_clean['Marxistisch-Leninistische Partei Deutschlands'].astype('int64') +\
                                   elections_2017_df_clean['Bürgerrechtsbewegung Solidarität'].astype('int64') +\
                                   elections_2017_df_clean['Sozialistische Gleichheitspartei, Vierte Internationale'].astype('int64') +\
                                   elections_2017_df_clean['DIE RECHTE'].astype('int64') +\
                                   elections_2017_df_clean['Allianz Deutscher Demokraten'].astype('int64') +\
                                   elections_2017_df_clean['Allianz für Menschenrechte, Tier- und Naturschutz'].astype('int64') +\
                                   elections_2017_df_clean['bergpartei, die überpartei'].astype('int64') +\
                                   elections_2017_df_clean['Bündnis Grundeinkommen'].astype('int64') +\
                                   elections_2017_df_clean['DEMOKRATIE IN BEWEGUNG'].astype('int64') +\
                                   elections_2017_df_clean['Deutsche Mitte'].astype('int64') +\
                                   elections_2017_df_clean['Die Grauen – Für alle Generationen'].astype('int64') +\
                                   elections_2017_df_clean['Die Urbane. Eine HipHop Partei'].astype('int64') +\
                                   elections_2017_df_clean['Magdeburger Gartenpartei'].astype('int64') +\
                                   elections_2017_df_clean['Menschliche Welt'].astype('int64') +\
                                   elections_2017_df_clean['Partei der Humanisten'].astype('int64') +\
                                   elections_2017_df_clean['Partei für Gesundheitsforschung'].astype('int64') +\
                                   elections_2017_df_clean['V-Partei³ - Partei für Veränderung, Vegetarier und Veganer'].astype('int64') +\
                                   elections_2017_df_clean['Bündnis C - Christen für Deutschland'].astype('int64') +\
                                   elections_2017_df_clean['DIE EINHEIT'].astype('int64') +\
                                   elections_2017_df_clean['Die Violetten'].astype('int64') +\
                                   elections_2017_df_clean['Familien-Partei Deutschlands'].astype('int64') +\
                                   elections_2017_df_clean['Feministische Partei DIE FRAUEN'].astype('int64') +\
                                   elections_2017_df_clean['Mieterpartei'].astype('int64') +\
                                   elections_2017_df_clean['Neue Liberale – Die Sozialliberalen'].astype('int64') +\
                                   elections_2017_df_clean['UNABHÄNGIGE für bürgernahe Demokratie'].astype('int64') +\
                                   elections_2017_df_clean['Übrige'].astype('int64')

#others previous election results
elections_2017_df_final['OTHER_prev'] = elections_2017_df_clean['Unnamed: 48'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 52'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 56'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 60'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 64'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 68'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 72'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 76'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 80'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 84'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 88'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 92'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 96'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 100'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 104'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 108'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 112'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 116'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 124'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 128'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 132'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 136'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 140'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 144'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 148'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 152'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 156'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 160'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 164'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 168'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 172'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 176'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 180'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 184'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 188'].astype('int64') 

#others Zweitstimmen
elections_2017_df_final['OTHER:2'] =    elections_2017_df_clean['Unnamed: 49'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 53'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 57'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 61'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 65'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 69'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 73'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 77'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 81'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 85'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 89'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 93'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 97'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 101'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 105'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 109'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 113'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 117'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 125'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 129'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 133'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 137'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 141'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 145'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 149'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 153'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 157'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 161'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 165'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 169'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 173'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 177'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 181'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 185'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 189'].astype('int64') 

#others Zweitstimmen previous election
elections_2017_df_final['OTHER:2_prev'] = elections_2017_df_clean['Unnamed: 50'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 54'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 58'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 62'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 66'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 70'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 74'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 78'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 82'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 86'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 90'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 94'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 98'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 102'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 106'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 110'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 114'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 118'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 126'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 130'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 134'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 138'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 142'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 146'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 150'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 154'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 158'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 162'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 166'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 170'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 174'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 178'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 182'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 186'].astype('int64') + \
                                        elections_2017_df_clean['Unnamed: 190'].astype('int64')  

#fill NaNs with 0s
elections_2017_df_final = elections_2017_df_final.fillna(0)

#double-check that all votes have been counted
A = elections_2017_df_final['Gültige'] - elections_2017_df_final['CDU/CSU'] - elections_2017_df_final['SPD'] \
  - elections_2017_df_final['AFD'] - elections_2017_df_final['FDP'] - elections_2017_df_final['GRUENE'] \
  - elections_2017_df_final['KPD'] - elections_2017_df_final['LINKE'] 
B = elections_2017_df_final['OTHER']
assert(np.allclose(A, B))

#double-ckeck that all second votes are counted correctly
A2 = elections_2017_df_final['Gültige:2'] - elections_2017_df_final['CDU/CSU:2'] - elections_2017_df_final['SPD:2'] \
   - elections_2017_df_final['AFD:2'] - elections_2017_df_final['FDP:2'] - elections_2017_df_final['GRUENE:2'] \
   - elections_2017_df_final['KPD:2'] - elections_2017_df_final['LINKE:2'] 
B2 = elections_2017_df_final['OTHER:2']
assert(np.allclose(A2, B2))

elections_2017_df_final.to_csv('Bundestagswahlen_clean/elections_2017.csv')